**This notebook contains the executed random-stratified experiments for all nine**. **models, including evaluation metrics, ROC curves, confusion matrices and saved predictions**.

In [ ]:
!pip install seaborn
import seaborn as sns
import matplotlib.pyplot as plt

Defaulting to user installation because normal site-packages is not writeable


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added random stratified test split for robustness evaluation
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
"""

import os
import random
import numpy as npA
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 780
EPOCHS_CV = 200
BATCH_CV = 24
LR_CV = 0.00012
EARLY_STOP_PR = 0.90

BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "LEFT_EYE_1_eye_stratified_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# CREATE RANDOM STRATIFIED SPLIT (Training + Validation + Test)
# =========================
print("\n--- Creating Random Stratified Split (Training + Validation + Test) ---")
# Combine all data for stratified split
all_imgs = train_imgs + val_imgs + test_imgs
all_lbls = train_lbls + val_lbls + test_lbls
all_filenames = train_filenames + val_filenames + test_filenames

# Create stratified random splits - 75% train, 12.5% val, 12.5% test
X_temp, X_test_strat, y_temp, y_test_strat, fname_temp, fname_test_strat = train_test_split(
    all_imgs, all_lbls, all_filenames,
    test_size=0.125,  # 12.5% for test
    stratify=all_lbls,
    random_state=SEED
)

X_train_strat, X_val_strat, y_train_strat, y_val_strat, fname_train_strat, fname_val_strat = train_test_split(
    X_temp, y_temp, fname_temp,
    test_size=0.142857,  # 12.5% of total (14.2857% of remaining 87.5%) for validation
    stratify=y_temp,
    random_state=SEED
)

print(f"Random stratified train set: {sum(y_train_strat)} anemic / {len(y_train_strat)} total")
print(f"Random stratified val set: {sum(y_val_strat)} anemic / {len(y_val_strat)} total")
print(f"Random stratified test set: {sum(y_test_strat)} anemic / {len(y_test_strat)} total")
print(f"Original test set: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS AND PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []

    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])

    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())

    if len(set(labels_all)) < 2:
        return float("nan"), float("nan"), float("nan"), float("nan"), float("nan"), 0, 0, 0, 0, labels_all, probs, all_filenames, preds

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()
    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc

    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt

    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'],
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'],
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1],
                   tflite_metrics[2], tflite_metrics[3],
                   tflite_metrics[4]]

    x = np.arange(len(metrics))
    width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')

    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    # Use random stratified training data for CV
    X = X_train_strat  # Random stratified training data
    y = y_train_strat
    filenames = fname_train_strat

    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []

    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")

        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]

        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()

            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")

            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR and P <1 and R < 1:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")

        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)

    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]

    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]

    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Random stratified test evaluation
    test_loader_strat = DataLoader(
        SingleEyeDataset(X_test_strat, y_test_strat, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])
    test_metrics_strat = evaluate_with_predictions(model, test_loader_strat, fname_test_strat)

    print("\n📊 FINAL TEST RESULTS (Random Stratified Test Set):")
    print(f"Precision: {test_metrics_strat[0]:.4f}")
    print(f"Recall:    {test_metrics_strat[1]:.4f}")
    print(f"F1 score:  {test_metrics_strat[2]:.4f}")
    print(f"Accuracy:  {test_metrics_strat[3]:.4f}")
    print(f"AUC:       {test_metrics_strat[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics_strat[5])}, {int(test_metrics_strat[6])}, {int(test_metrics_strat[7])}, {int(test_metrics_strat[8])}")


    # Save stratified test results
    _strat_row = [{
         'Test_Precision': test_metrics_strat[0],
         'Test_Recall': test_metrics_strat[1],
         'Test_F1': test_metrics_strat[2],
         'Test_Accuracy': test_metrics_strat[3],
         'Test_AUC': test_metrics_strat[4],
         'Test_TP': test_metrics_strat[5],
         'Test_TN': test_metrics_strat[6],
         'Test_FP': test_metrics_strat[7],
         'Test_FN': test_metrics_strat[8],
         'Best_Fold': best_fold
    }]
    _strat_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_strat_row)[_strat_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results_stratified.csv"), index=False)

    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics_strat[11],  # filenames
        test_metrics_strat[9],   # true labels
        test_metrics_strat[12],  # pred labels
        test_metrics_strat[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch.csv")
    )

    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics_strat[9], test_metrics_strat[10],
                   "ROC Curve - PyTorch Model (Random Stratified Test Set)",
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics_strat[9],
                          test_metrics_strat[12],
                          "Confusion Matrix - PyTorch Model",
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))

    return model, test_metrics_strat

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)

    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE)
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")

    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce:
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    model, strat_test_metrics = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")

    # Convert to TFLite
    convert_to_tflite(model, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on random stratified test set
    print("\n🔍 Loading TFLite model and re-evaluating on random stratified test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics_strat = evaluate_tflite_on_test_with_predictions(tflite_file, X_test_strat, y_test_strat, fname_test_strat)

    if tflite_metrics_strat:
        P_strat, R_strat, F1_strat, acc_strat, auc_strat, tp_strat, tn_strat, fp_strat, fn_strat, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics_strat
        print("\n📊 TFLITE TEST RESULTS (Random Stratified Test Set):")
        print(f"Precision: {P_strat:.4f}")
        print(f"Recall:    {R_strat:.4f}")
        print(f"F1 score:  {F1_strat:.4f}")
        print(f"Accuracy:  {acc_strat:.4f}")
        print(f"AUC:       {auc_strat:.4f}")
        print(f"TP, TN, FP, FN: {int(tp_strat)}, {int(tn_strat)}, {int(fp_strat)}, {int(fn_strat)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs,
                       "ROC Curve - TFLite Model (Random Stratified Test Set)",
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels,
                              tflite_preds,
                              "Confusion Matrix - TFLite Model",
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        # Compare with original PyTorch test results
        test_results_path = os.path.join(OUTPUT_DIR, "test_results_stratified.csv")
        if os.path.exists(test_results_path):
            strat_pytorch = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results (Random Stratified):")
            print(f"PyTorch → P: {strat_pytorch['Test_Precision']:.4f}, R: {strat_pytorch['Test_Recall']:.4f}, AUC: {strat_pytorch['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P_strat:.4f}, R: {R_strat:.4f}, AUC: {auc_strat:.4f}")

            tol = 1e-3
            p_ok = abs(P_strat - strat_pytorch['Test_Precision']) < tol
            r_ok = abs(R_strat - strat_pytorch['Test_Recall']) < tol
            auc_ok = abs(auc_strat - strat_pytorch['Test_AUC']) < tol

            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")

            # Create metrics comparison plot
            plot_metrics_comparison(strat_pytorch, tflite_metrics_strat,
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed on random stratified test set.")

    print("\n✅ Analysis complete: Random stratified test set evaluated with comprehensive plots and detailed CSV predictions!")
    print(f"✅ Detailed prediction CSVs saved to: {OUTPUT_DIR}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 44 anemic / 358 total

--- Creating Random Stratified Split (Training + Validation + Test) ---
Random stratified train set: 255 anemic / 2152 total
Random stratified val set: 42 anemic / 359 total
Random stratified test set: 42 anemic / 359 total
Original test set: 44 anemic / 358 total

--- Fold 1 ---
Epoch 20/200 Loss: 8.3448
Epoch 40/200 Loss: 1.4184
Epoch 60/200 Loss: 0.5800
Epoch 80/200 Loss: 0.4744
Epoch 100/200 Loss: 0.0108
✅ Early stop at epoch 112: P=0.979, R=0.902
Fold 1 → P=0.979, R=0.902

--- Fold 2 ---
Epoch 20/200 Loss: 0.7714
Epoch 40/200 Loss: 0.0629
✅ Early stop at epoch 50: P=0.980, R=0.961
Fold 2 → P=0.980, R=0.961

--- Fold 3 ---
Epoch 20/200 Loss: 1.1884
✅ Early stop at epoch 20: P=0.980, R=0.980
Fold 3 → P=0.980, R=0.980

--- Fold 4 ---
Epoch 20/200 Loss: 0.4454
Epoch 40/200 Loss: 0.0626
✅ Early stop at epoch 49: P=0.979, R=0.902
Fold 4 → P=0.979, R=0.902

--- Fold 5 ---
Epoch 20/200 Loss: 4.6756
✅ Early stop at 

2025-11-16 23:24:59.004455: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-16 23:24:59.006045: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-16 23:24:59.035577: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-16 23:24:59.608677: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



--- Starting TFLite Conversion Pipeline ---
1. Converting PyTorch model to ONNX...
   ✅ ONNX model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_stratified_repro/model.onnx
2. Converting ONNX model to TensorFlow SavedModel...


2025-11-16 23:25:01.342085: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-16 23:25:01.343716: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will 

INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_stratified_repro/tf_model/assets


...

   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_stratified_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-11-16 23:25:06.458868: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-16 23:25:06.458902: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-16 23:25:06.461261: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_stratified_repro/tf_model
2025-11-16 23:25:06.495943: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-16 23:25:06.495963: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_stratified_repro/tf_model
2025-11-16 23:25:06.527376: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-11-16 23:25:06.528079: I tensorflow/

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_stratified_repro/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on random stratified test set...


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Random Stratified Test Set):
Precision: 0.9750
Recall:    0.9286
F1 score:  0.9512
Accuracy:  0.9889
AUC:       0.9994
TP, TN, FP, FN: 39, 316, 1, 3
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_stratified_repro/detailed_predictions_tflite.csv

🔍 Comparing with original PyTorch test results (Random Stratified):
PyTorch → P: 0.9750, R: 0.9286, AUC: 0.9994
TFLite  → P: 0.9750, R: 0.9286, AUC: 0.9994
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Random stratified test set evaluated with comprehensive plots and detailed CSV predictions!
✅ Detailed prediction CSVs saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_stratified_repro


In [ ]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added random stratified test split for robustness evaluation
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 780
EPOCHS_CV = 200
BATCH_CV = 16
LR_CV = 0.00003
EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90


BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "Right_EYE_2_eye_stratified_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# CREATE RANDOM STRATIFIED SPLIT (Training + Validation + Test)
# =========================
print("\n--- Creating Random Stratified Split (Training + Validation + Test) ---")
# Combine all data for stratified split
all_imgs = train_imgs + val_imgs + test_imgs
all_lbls = train_lbls + val_lbls + test_lbls
all_filenames = train_filenames + val_filenames + test_filenames

# Create stratified random splits - 75% train, 12.5% val, 12.5% test
X_temp, X_test_strat, y_temp, y_test_strat, fname_temp, fname_test_strat = train_test_split(
    all_imgs, all_lbls, all_filenames,
    test_size=0.125,  # 12.5% for test
    stratify=all_lbls,
    random_state=SEED
)

X_train_strat, X_val_strat, y_train_strat, y_val_strat, fname_train_strat, fname_val_strat = train_test_split(
    X_temp, y_temp, fname_temp,
    test_size=0.142857,  # 12.5% of total (14.2857% of remaining 87.5%) for validation
    stratify=y_temp,
    random_state=SEED
)

print(f"Random stratified train set: {sum(y_train_strat)} anemic / {len(y_train_strat)} total")
print(f"Random stratified val set: {sum(y_val_strat)} anemic / {len(y_val_strat)} total")
print(f"Random stratified test set: {sum(y_test_strat)} anemic / {len(y_test_strat)} total")
print(f"Original test set: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS AND PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []

    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])

    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())

    if len(set(labels_all)) < 2:
        return float("nan"), float("nan"), float("nan"), float("nan"), float("nan"), 0, 0, 0, 0, labels_all, probs, all_filenames, preds

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()
    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc

    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt

    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'],
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'],
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1],
                   tflite_metrics[2], tflite_metrics[3],
                   tflite_metrics[4]]

    x = np.arange(len(metrics))
    width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')

    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    # Use random stratified training data for CV
    X = X_train_strat  # Random stratified training data
    y = y_train_strat
    filenames = fname_train_strat

    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []

    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")

        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]

        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()

            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")

            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR and P <1 and R < 1:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")

        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)

    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]

    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]

    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Random stratified test evaluation
    test_loader_strat = DataLoader(
        SingleEyeDataset(X_test_strat, y_test_strat, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])
    test_metrics_strat = evaluate_with_predictions(model, test_loader_strat, fname_test_strat)

    print("\n📊 FINAL TEST RESULTS (Random Stratified Test Set):")
    print(f"Precision: {test_metrics_strat[0]:.4f}")
    print(f"Recall:    {test_metrics_strat[1]:.4f}")
    print(f"F1 score:  {test_metrics_strat[2]:.4f}")
    print(f"Accuracy:  {test_metrics_strat[3]:.4f}")
    print(f"AUC:       {test_metrics_strat[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics_strat[5])}, {int(test_metrics_strat[6])}, {int(test_metrics_strat[7])}, {int(test_metrics_strat[8])}")


    # Save stratified test results
    _strat_row = [{
         'Test_Precision': test_metrics_strat[0],
         'Test_Recall': test_metrics_strat[1],
         'Test_F1': test_metrics_strat[2],
         'Test_Accuracy': test_metrics_strat[3],
         'Test_AUC': test_metrics_strat[4],
         'Test_TP': test_metrics_strat[5],
         'Test_TN': test_metrics_strat[6],
         'Test_FP': test_metrics_strat[7],
         'Test_FN': test_metrics_strat[8],
         'Best_Fold': best_fold
    }]
    _strat_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_strat_row)[_strat_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results_stratified.csv"), index=False)

    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics_strat[11],  # filenames
        test_metrics_strat[9],   # true labels
        test_metrics_strat[12],  # pred labels
        test_metrics_strat[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch.csv")
    )

    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics_strat[9], test_metrics_strat[10],
                   "ROC Curve - PyTorch Model (Random Stratified Test Set)",
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics_strat[9],
                          test_metrics_strat[12],
                          "Confusion Matrix - PyTorch Model",
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))

    return model, test_metrics_strat

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)

    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE)
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")

    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce:
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    model, strat_test_metrics = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")

    # Convert to TFLite
    convert_to_tflite(model, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on random stratified test set
    print("\n🔍 Loading TFLite model and re-evaluating on random stratified test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics_strat = evaluate_tflite_on_test_with_predictions(tflite_file, X_test_strat, y_test_strat, fname_test_strat)

    if tflite_metrics_strat:
        P_strat, R_strat, F1_strat, acc_strat, auc_strat, tp_strat, tn_strat, fp_strat, fn_strat, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics_strat
        print("\n📊 TFLITE TEST RESULTS (Random Stratified Test Set):")
        print(f"Precision: {P_strat:.4f}")
        print(f"Recall:    {R_strat:.4f}")
        print(f"F1 score:  {F1_strat:.4f}")
        print(f"Accuracy:  {acc_strat:.4f}")
        print(f"AUC:       {auc_strat:.4f}")
        print(f"TP, TN, FP, FN: {int(tp_strat)}, {int(tn_strat)}, {int(fp_strat)}, {int(fn_strat)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs,
                       "ROC Curve - TFLite Model (Random Stratified Test Set)",
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels,
                              tflite_preds,
                              "Confusion Matrix - TFLite Model",
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        # Compare with original PyTorch test results
        test_results_path = os.path.join(OUTPUT_DIR, "test_results_stratified.csv")
        if os.path.exists(test_results_path):
            strat_pytorch = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results (Random Stratified):")
            print(f"PyTorch → P: {strat_pytorch['Test_Precision']:.4f}, R: {strat_pytorch['Test_Recall']:.4f}, AUC: {strat_pytorch['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P_strat:.4f}, R: {R_strat:.4f}, AUC: {auc_strat:.4f}")

            tol = 1e-3
            p_ok = abs(P_strat - strat_pytorch['Test_Precision']) < tol
            r_ok = abs(R_strat - strat_pytorch['Test_Recall']) < tol
            auc_ok = abs(auc_strat - strat_pytorch['Test_AUC']) < tol

            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")

            # Create metrics comparison plot
            plot_metrics_comparison(strat_pytorch, tflite_metrics_strat,
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed on random stratified test set.")

    print("\n✅ Analysis complete: Random stratified test set evaluated with comprehensive plots and detailed CSV predictions!")
    print(f"✅ Detailed prediction CSVs saved to: {OUTPUT_DIR}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 40 anemic / 369 total

--- Creating Random Stratified Split (Training + Validation + Test) ---
Random stratified train set: 255 anemic / 2218 total
Random stratified val set: 42 anemic / 370 total
Random stratified test set: 43 anemic / 370 total
Original test set: 40 anemic / 369 total

--- Fold 1 ---
✅ Early stop at epoch 15: P=0.920, R=0.902
Fold 1 → P=0.920, R=0.902

--- Fold 2 ---
Epoch 20/200 Loss: 1.1620
Epoch 40/200 Loss: 0.1085
Epoch 60/200 Loss: 0.0590
Epoch 80/200 Loss: 0.0180
Epoch 100/200 Loss: 0.4234
Epoch 120/200 Loss: 0.0252
Epoch 140/200 Loss: 0.0093
Epoch 160/200 Loss: 0.0053
Epoch 180/200 Loss: 0.1676
Epoch 200/200 Loss: 0.0139
Fold 2 → P=1.000, R=1.000

--- Fold 3 ---
Epoch 20/200 Loss: 2.1314
✅ Early stop at epoch 30: P=0.980, R=0.941
Fold 3 → P=0.980, R=0.941

--- Fold 4 ---
✅ Early stop at epoch 15: P=0.980, R=0.941
Fold 4 → P=0.980, R=0.941

--- Fold 5 ---
Epoch 20/200 Loss: 2.5919
✅ Early stop at epoch 24: P=0

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_2_eye_stratified_repro/tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_2_eye_stratified_repro/tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_2_eye_stratified_repro/tf_model/fingerprint.pb


   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_2_eye_stratified_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-11-17 01:27:14.554526: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-17 01:27:14.554561: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-17 01:27:14.555917: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_2_eye_stratified_repro/tf_model
2025-11-17 01:27:14.588360: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-17 01:27:14.588381: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_2_eye_stratified_repro/tf_model
2025-11-17 01:27:14.598847: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-11-17 01:27:14.621746: I tensorflow/cc/saved_model/loader.cc:215] R

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_2_eye_stratified_repro/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on random stratified test set...
🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Random Stratified Test Set):
Precision: 1.0000
Recall:    1.0000
F1 score:  1.0000
Accuracy:  1.0000
AUC:       1.0000
TP, TN, FP, FN: 43, 327, 0, 0
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_2_eye_stratified_repro/detailed_predictions_tflite.csv

🔍 Comparing with original PyTorch test results (Random Stratified):
PyTorch → P: 1.0000, R: 1.0000, AUC: 1.0000
TFLite  → P: 1.0000, R: 1.0000, AUC: 1.0000
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Random stratified test set evaluate

In [ ]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added random stratified test split for robustness evaluation
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 780
EPOCHS_CV = 200
BATCH_CV = 8
LR_CV = 0.00025

EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90

BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "LEFT_EYE_2_eye_stratified_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)
# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# CREATE RANDOM STRATIFIED SPLIT (Training + Validation + Test)
# =========================
print("\n--- Creating Random Stratified Split (Training + Validation + Test) ---")
# Combine all data for stratified split
all_imgs = train_imgs + val_imgs + test_imgs
all_lbls = train_lbls + val_lbls + test_lbls
all_filenames = train_filenames + val_filenames + test_filenames

# Create stratified random splits - 75% train, 12.5% val, 12.5% test
X_temp, X_test_strat, y_temp, y_test_strat, fname_temp, fname_test_strat = train_test_split(
    all_imgs, all_lbls, all_filenames,
    test_size=0.125,  # 12.5% for test
    stratify=all_lbls,
    random_state=SEED
)

X_train_strat, X_val_strat, y_train_strat, y_val_strat, fname_train_strat, fname_val_strat = train_test_split(
    X_temp, y_temp, fname_temp,
    test_size=0.142857,  # 12.5% of total (14.2857% of remaining 87.5%) for validation
    stratify=y_temp,
    random_state=SEED
)

print(f"Random stratified train set: {sum(y_train_strat)} anemic / {len(y_train_strat)} total")
print(f"Random stratified val set: {sum(y_val_strat)} anemic / {len(y_val_strat)} total")
print(f"Random stratified test set: {sum(y_test_strat)} anemic / {len(y_test_strat)} total")
print(f"Original test set: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS AND PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []

    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])

    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())

    if len(set(labels_all)) < 2:
        return float("nan"), float("nan"), float("nan"), float("nan"), float("nan"), 0, 0, 0, 0, labels_all, probs, all_filenames, preds

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()
    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc

    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt

    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'],
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'],
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1],
                   tflite_metrics[2], tflite_metrics[3],
                   tflite_metrics[4]]

    x = np.arange(len(metrics))
    width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')

    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    # Use random stratified training data for CV
    X = X_train_strat  # Random stratified training data
    y = y_train_strat
    filenames = fname_train_strat

    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []

    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")

        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]

        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()

            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")

            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR and P <1 and R < 1:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")

        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)

    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]

    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]

    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Random stratified test evaluation
    test_loader_strat = DataLoader(
        SingleEyeDataset(X_test_strat, y_test_strat, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])
    test_metrics_strat = evaluate_with_predictions(model, test_loader_strat, fname_test_strat)

    print("\n📊 FINAL TEST RESULTS (Random Stratified Test Set):")
    print(f"Precision: {test_metrics_strat[0]:.4f}")
    print(f"Recall:    {test_metrics_strat[1]:.4f}")
    print(f"F1 score:  {test_metrics_strat[2]:.4f}")
    print(f"Accuracy:  {test_metrics_strat[3]:.4f}")
    print(f"AUC:       {test_metrics_strat[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics_strat[5])}, {int(test_metrics_strat[6])}, {int(test_metrics_strat[7])}, {int(test_metrics_strat[8])}")


    # Save stratified test results
    _strat_row = [{
         'Test_Precision': test_metrics_strat[0],
         'Test_Recall': test_metrics_strat[1],
         'Test_F1': test_metrics_strat[2],
         'Test_Accuracy': test_metrics_strat[3],
         'Test_AUC': test_metrics_strat[4],
         'Test_TP': test_metrics_strat[5],
         'Test_TN': test_metrics_strat[6],
         'Test_FP': test_metrics_strat[7],
         'Test_FN': test_metrics_strat[8],
         'Best_Fold': best_fold
    }]
    _strat_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_strat_row)[_strat_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results_stratified.csv"), index=False)

    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics_strat[11],  # filenames
        test_metrics_strat[9],   # true labels
        test_metrics_strat[12],  # pred labels
        test_metrics_strat[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch.csv")
    )

    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics_strat[9], test_metrics_strat[10],
                   "ROC Curve - PyTorch Model (Random Stratified Test Set)",
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics_strat[9],
                          test_metrics_strat[12],
                          "Confusion Matrix - PyTorch Model",
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))

    return model, test_metrics_strat

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)

    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE)
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")

    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce:
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    model, strat_test_metrics = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")

    # Convert to TFLite
    convert_to_tflite(model, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on random stratified test set
    print("\n🔍 Loading TFLite model and re-evaluating on random stratified test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics_strat = evaluate_tflite_on_test_with_predictions(tflite_file, X_test_strat, y_test_strat, fname_test_strat)

    if tflite_metrics_strat:
        P_strat, R_strat, F1_strat, acc_strat, auc_strat, tp_strat, tn_strat, fp_strat, fn_strat, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics_strat
        print("\n📊 TFLITE TEST RESULTS (Random Stratified Test Set):")
        print(f"Precision: {P_strat:.4f}")
        print(f"Recall:    {R_strat:.4f}")
        print(f"F1 score:  {F1_strat:.4f}")
        print(f"Accuracy:  {acc_strat:.4f}")
        print(f"AUC:       {auc_strat:.4f}")
        print(f"TP, TN, FP, FN: {int(tp_strat)}, {int(tn_strat)}, {int(fp_strat)}, {int(fn_strat)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs,
                       "ROC Curve - TFLite Model (Random Stratified Test Set)",
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels,
                              tflite_preds,
                              "Confusion Matrix - TFLite Model",
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        # Compare with original PyTorch test results
        test_results_path = os.path.join(OUTPUT_DIR, "test_results_stratified.csv")
        if os.path.exists(test_results_path):
            strat_pytorch = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results (Random Stratified):")
            print(f"PyTorch → P: {strat_pytorch['Test_Precision']:.4f}, R: {strat_pytorch['Test_Recall']:.4f}, AUC: {strat_pytorch['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P_strat:.4f}, R: {R_strat:.4f}, AUC: {auc_strat:.4f}")

            tol = 1e-3
            p_ok = abs(P_strat - strat_pytorch['Test_Precision']) < tol
            r_ok = abs(R_strat - strat_pytorch['Test_Recall']) < tol
            auc_ok = abs(auc_strat - strat_pytorch['Test_AUC']) < tol

            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")

            # Create metrics comparison plot
            plot_metrics_comparison(strat_pytorch, tflite_metrics_strat,
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed on random stratified test set.")

    print("\n✅ Analysis complete: Random stratified test set evaluated with comprehensive plots and detailed CSV predictions!")
    print(f"✅ Detailed prediction CSVs saved to: {OUTPUT_DIR}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 42 anemic / 361 total

--- Creating Random Stratified Split (Training + Validation + Test) ---
Random stratified train set: 254 anemic / 2169 total
Random stratified val set: 42 anemic / 362 total
Random stratified test set: 42 anemic / 362 total
Original test set: 42 anemic / 361 total

--- Fold 1 ---
✅ Early stop at epoch 11: P=0.980, R=0.980
Fold 1 → P=0.980, R=0.980

--- Fold 2 ---
✅ Early stop at epoch 4: P=0.980, R=0.980
Fold 2 → P=0.980, R=0.980

--- Fold 3 ---
Epoch 20/200 Loss: 3.5058
Epoch 40/200 Loss: 3.4240
Epoch 60/200 Loss: 0.7244
✅ Early stop at epoch 66: P=0.980, R=0.961
Fold 3 → P=0.980, R=0.961

--- Fold 4 ---
✅ Early stop at epoch 3: P=0.980, R=0.941
Fold 4 → P=0.980, R=0.941

--- Fold 5 ---
✅ Early stop at epoch 3: P=0.961, R=0.980
Fold 5 → P=0.961, R=0.980
✅ Best fold = 2 | P=0.980, R=0.980

📊 FINAL TEST RESULTS (Random Stratified Test Set):
Precision: 0.9500
Recall:    0.9048
F1 score:  0.9268
Accuracy:  0.9834
A

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_stratified_repro/tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_stratified_repro/tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_stratified_repro/tf_model/fingerprint.pb


   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_stratified_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-11-17 02:06:00.416153: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-17 02:06:00.416189: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-17 02:06:00.417951: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_stratified_repro/tf_model
2025-11-17 02:06:00.450036: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-17 02:06:00.450058: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_stratified_repro/tf_model
2025-11-17 02:06:00.461402: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-11-17 02:06:00.483765: I tensorflow/cc/saved_model/loader.cc:215] Run

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_stratified_repro/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on random stratified test set...
🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Random Stratified Test Set):
Precision: 0.9500
Recall:    0.9048
F1 score:  0.9268
Accuracy:  0.9834
AUC:       0.9984
TP, TN, FP, FN: 38, 318, 2, 4
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_stratified_repro/detailed_predictions_tflite.csv

🔍 Comparing with original PyTorch test results (Random Stratified):
PyTorch → P: 0.9500, R: 0.9048, AUC: 0.9984
TFLite  → P: 0.9500, R: 0.9048, AUC: 0.9984
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Random stratified test set evaluated 

In [ ]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added random stratified test split for robustness evaluation
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 780
EPOCHS_CV = 200
BATCH_CV = 32
LR_CV = 0.000003
EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90

BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "Right_EYE_3_eye_stratified_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# CREATE RANDOM STRATIFIED SPLIT (Training + Validation + Test)
# =========================
print("\n--- Creating Random Stratified Split (Training + Validation + Test) ---")
# Combine all data for stratified split
all_imgs = train_imgs + val_imgs + test_imgs
all_lbls = train_lbls + val_lbls + test_lbls
all_filenames = train_filenames + val_filenames + test_filenames

# Create stratified random splits - 75% train, 12.5% val, 12.5% test
X_temp, X_test_strat, y_temp, y_test_strat, fname_temp, fname_test_strat = train_test_split(
    all_imgs, all_lbls, all_filenames,
    test_size=0.125,  # 12.5% for test
    stratify=all_lbls,
    random_state=SEED
)

X_train_strat, X_val_strat, y_train_strat, y_val_strat, fname_train_strat, fname_val_strat = train_test_split(
    X_temp, y_temp, fname_temp,
    test_size=0.142857,  # 12.5% of total (14.2857% of remaining 87.5%) for validation
    stratify=y_temp,
    random_state=SEED
)

print(f"Random stratified train set: {sum(y_train_strat)} anemic / {len(y_train_strat)} total")
print(f"Random stratified val set: {sum(y_val_strat)} anemic / {len(y_val_strat)} total")
print(f"Random stratified test set: {sum(y_test_strat)} anemic / {len(y_test_strat)} total")
print(f"Original test set: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS AND PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []

    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])

    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())

    if len(set(labels_all)) < 2:
        return float("nan"), float("nan"), float("nan"), float("nan"), float("nan"), 0, 0, 0, 0, labels_all, probs, all_filenames, preds

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()
    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc

    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt

    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'],
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'],
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1],
                   tflite_metrics[2], tflite_metrics[3],
                   tflite_metrics[4]]

    x = np.arange(len(metrics))
    width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')

    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    # Use random stratified training data for CV
    X = X_train_strat  # Random stratified training data
    y = y_train_strat
    filenames = fname_train_strat

    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []

    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")

        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]

        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()

            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")

            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR and P <1 and R < 1:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")

        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)

    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]

    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]

    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Random stratified test evaluation
    test_loader_strat = DataLoader(
        SingleEyeDataset(X_test_strat, y_test_strat, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])
    test_metrics_strat = evaluate_with_predictions(model, test_loader_strat, fname_test_strat)

    print("\n📊 FINAL TEST RESULTS (Random Stratified Test Set):")
    print(f"Precision: {test_metrics_strat[0]:.4f}")
    print(f"Recall:    {test_metrics_strat[1]:.4f}")
    print(f"F1 score:  {test_metrics_strat[2]:.4f}")
    print(f"Accuracy:  {test_metrics_strat[3]:.4f}")
    print(f"AUC:       {test_metrics_strat[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics_strat[5])}, {int(test_metrics_strat[6])}, {int(test_metrics_strat[7])}, {int(test_metrics_strat[8])}")


    # Save stratified test results
    _strat_row = [{
         'Test_Precision': test_metrics_strat[0],
         'Test_Recall': test_metrics_strat[1],
         'Test_F1': test_metrics_strat[2],
         'Test_Accuracy': test_metrics_strat[3],
         'Test_AUC': test_metrics_strat[4],
         'Test_TP': test_metrics_strat[5],
         'Test_TN': test_metrics_strat[6],
         'Test_FP': test_metrics_strat[7],
         'Test_FN': test_metrics_strat[8],
         'Best_Fold': best_fold
    }]
    _strat_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_strat_row)[_strat_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results_stratified.csv"), index=False)

    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics_strat[11],  # filenames
        test_metrics_strat[9],   # true labels
        test_metrics_strat[12],  # pred labels
        test_metrics_strat[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch.csv")
    )

    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics_strat[9], test_metrics_strat[10],
                   "ROC Curve - PyTorch Model (Random Stratified Test Set)",
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics_strat[9],
                          test_metrics_strat[12],
                          "Confusion Matrix - PyTorch Model",
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))

    return model, test_metrics_strat

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)

    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE)
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")

    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce:
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    model, strat_test_metrics = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")

    # Convert to TFLite
    convert_to_tflite(model, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on random stratified test set
    print("\n🔍 Loading TFLite model and re-evaluating on random stratified test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics_strat = evaluate_tflite_on_test_with_predictions(tflite_file, X_test_strat, y_test_strat, fname_test_strat)

    if tflite_metrics_strat:
        P_strat, R_strat, F1_strat, acc_strat, auc_strat, tp_strat, tn_strat, fp_strat, fn_strat, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics_strat
        print("\n📊 TFLITE TEST RESULTS (Random Stratified Test Set):")
        print(f"Precision: {P_strat:.4f}")
        print(f"Recall:    {R_strat:.4f}")
        print(f"F1 score:  {F1_strat:.4f}")
        print(f"Accuracy:  {acc_strat:.4f}")
        print(f"AUC:       {auc_strat:.4f}")
        print(f"TP, TN, FP, FN: {int(tp_strat)}, {int(tn_strat)}, {int(fp_strat)}, {int(fn_strat)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs,
                       "ROC Curve - TFLite Model (Random Stratified Test Set)",
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels,
                              tflite_preds,
                              "Confusion Matrix - TFLite Model",
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        # Compare with original PyTorch test results
        test_results_path = os.path.join(OUTPUT_DIR, "test_results_stratified.csv")
        if os.path.exists(test_results_path):
            strat_pytorch = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results (Random Stratified):")
            print(f"PyTorch → P: {strat_pytorch['Test_Precision']:.4f}, R: {strat_pytorch['Test_Recall']:.4f}, AUC: {strat_pytorch['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P_strat:.4f}, R: {R_strat:.4f}, AUC: {auc_strat:.4f}")

            tol = 1e-3
            p_ok = abs(P_strat - strat_pytorch['Test_Precision']) < tol
            r_ok = abs(R_strat - strat_pytorch['Test_Recall']) < tol
            auc_ok = abs(auc_strat - strat_pytorch['Test_AUC']) < tol

            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")

            # Create metrics comparison plot
            plot_metrics_comparison(strat_pytorch, tflite_metrics_strat,
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed on random stratified test set.")

    print("\n✅ Analysis complete: Random stratified test set evaluated with comprehensive plots and detailed CSV predictions!")
    print(f"✅ Detailed prediction CSVs saved to: {OUTPUT_DIR}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 40 anemic / 368 total

--- Creating Random Stratified Split (Training + Validation + Test) ---
Random stratified train set: 254 anemic / 2211 total
Random stratified val set: 42 anemic / 369 total
Random stratified test set: 42 anemic / 369 total
Original test set: 40 anemic / 368 total

--- Fold 1 ---
Epoch 20/200 Loss: 17.9642
Epoch 40/200 Loss: 4.0380
Epoch 60/200 Loss: 1.5548
Epoch 80/200 Loss: 0.9898
Epoch 100/200 Loss: 0.4621
Epoch 120/200 Loss: 0.4131
Epoch 140/200 Loss: 0.2729
Epoch 160/200 Loss: 0.1374
Epoch 180/200 Loss: 0.1000
Epoch 200/200 Loss: 0.0980
Fold 1 → P=1.000, R=0.961

--- Fold 2 ---
Epoch 20/200 Loss: 18.4466
Epoch 40/200 Loss: 13.5051
✅ Early stop at epoch 50: P=0.978, R=0.900
Fold 2 → P=0.978, R=0.900

--- Fold 3 ---
Epoch 20/200 Loss: 18.6543
Epoch 40/200 Loss: 10.8715
Epoch 60/200 Loss: 4.5510
Epoch 80/200 Loss: 2.1403
Epoch 100/200 Loss: 1.1887
Epoch 120/200 Loss: 0.7236
Epoch 140/200 Loss: 0.3532
Epoch 160

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_3_eye_stratified_repro/tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_3_eye_stratified_repro/tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_3_eye_stratified_repro/tf_model/fingerprint.pb


   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_3_eye_stratified_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-11-17 06:41:17.858158: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-17 06:41:17.858192: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-17 06:41:17.859796: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_3_eye_stratified_repro/tf_model
2025-11-17 06:41:17.890992: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-17 06:41:17.891010: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_3_eye_stratified_repro/tf_model
2025-11-17 06:41:17.901555: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-11-17 06:41:17.923240: I tensorflow/cc/saved_model/loader.cc:215] R

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_3_eye_stratified_repro/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on random stratified test set...
🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Random Stratified Test Set):
Precision: 1.0000
Recall:    0.8810
F1 score:  0.9367
Accuracy:  0.9864
AUC:       1.0000
TP, TN, FP, FN: 37, 327, 0, 5
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_3_eye_stratified_repro/detailed_predictions_tflite.csv

🔍 Comparing with original PyTorch test results (Random Stratified):
PyTorch → P: 1.0000, R: 0.8810, AUC: 1.0000
TFLite  → P: 1.0000, R: 0.8810, AUC: 1.0000
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Random stratified test set evaluate

In [ ]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added random stratified test split for robustness evaluation
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 780
EPOCHS_CV = 200
BATCH_CV = 16
LR_CV = 0.0003

EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "LEFT_EYE_3_eye_stratified_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)


# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# CREATE RANDOM STRATIFIED SPLIT (Training + Validation + Test)
# =========================
print("\n--- Creating Random Stratified Split (Training + Validation + Test) ---")
# Combine all data for stratified split
all_imgs = train_imgs + val_imgs + test_imgs
all_lbls = train_lbls + val_lbls + test_lbls
all_filenames = train_filenames + val_filenames + test_filenames

# Create stratified random splits - 75% train, 12.5% val, 12.5% test
X_temp, X_test_strat, y_temp, y_test_strat, fname_temp, fname_test_strat = train_test_split(
    all_imgs, all_lbls, all_filenames,
    test_size=0.125,  # 12.5% for test
    stratify=all_lbls,
    random_state=SEED
)

X_train_strat, X_val_strat, y_train_strat, y_val_strat, fname_train_strat, fname_val_strat = train_test_split(
    X_temp, y_temp, fname_temp,
    test_size=0.142857,  # 12.5% of total (14.2857% of remaining 87.5%) for validation
    stratify=y_temp,
    random_state=SEED
)

print(f"Random stratified train set: {sum(y_train_strat)} anemic / {len(y_train_strat)} total")
print(f"Random stratified val set: {sum(y_val_strat)} anemic / {len(y_val_strat)} total")
print(f"Random stratified test set: {sum(y_test_strat)} anemic / {len(y_test_strat)} total")
print(f"Original test set: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS AND PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []

    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])

    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())

    if len(set(labels_all)) < 2:
        return float("nan"), float("nan"), float("nan"), float("nan"), float("nan"), 0, 0, 0, 0, labels_all, probs, all_filenames, preds

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()
    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc

    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt

    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'],
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'],
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1],
                   tflite_metrics[2], tflite_metrics[3],
                   tflite_metrics[4]]

    x = np.arange(len(metrics))
    width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')

    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    # Use random stratified training data for CV
    X = X_train_strat  # Random stratified training data
    y = y_train_strat
    filenames = fname_train_strat

    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []

    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")

        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]

        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()

            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")

            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR and P <1 and R < 1:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")

        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)

    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]

    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]

    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Random stratified test evaluation
    test_loader_strat = DataLoader(
        SingleEyeDataset(X_test_strat, y_test_strat, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])
    test_metrics_strat = evaluate_with_predictions(model, test_loader_strat, fname_test_strat)

    print("\n📊 FINAL TEST RESULTS (Random Stratified Test Set):")
    print(f"Precision: {test_metrics_strat[0]:.4f}")
    print(f"Recall:    {test_metrics_strat[1]:.4f}")
    print(f"F1 score:  {test_metrics_strat[2]:.4f}")
    print(f"Accuracy:  {test_metrics_strat[3]:.4f}")
    print(f"AUC:       {test_metrics_strat[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics_strat[5])}, {int(test_metrics_strat[6])}, {int(test_metrics_strat[7])}, {int(test_metrics_strat[8])}")


    # Save stratified test results
    _strat_row = [{
         'Test_Precision': test_metrics_strat[0],
         'Test_Recall': test_metrics_strat[1],
         'Test_F1': test_metrics_strat[2],
         'Test_Accuracy': test_metrics_strat[3],
         'Test_AUC': test_metrics_strat[4],
         'Test_TP': test_metrics_strat[5],
         'Test_TN': test_metrics_strat[6],
         'Test_FP': test_metrics_strat[7],
         'Test_FN': test_metrics_strat[8],
         'Best_Fold': best_fold
    }]
    _strat_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_strat_row)[_strat_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results_stratified.csv"), index=False)

    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics_strat[11],  # filenames
        test_metrics_strat[9],   # true labels
        test_metrics_strat[12],  # pred labels
        test_metrics_strat[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch.csv")
    )

    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics_strat[9], test_metrics_strat[10],
                   "ROC Curve - PyTorch Model (Random Stratified Test Set)",
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics_strat[9],
                          test_metrics_strat[12],
                          "Confusion Matrix - PyTorch Model",
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))

    return model, test_metrics_strat

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)

    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE)
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")

    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce:
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    model, strat_test_metrics = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")

    # Convert to TFLite
    convert_to_tflite(model, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on random stratified test set
    print("\n🔍 Loading TFLite model and re-evaluating on random stratified test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics_strat = evaluate_tflite_on_test_with_predictions(tflite_file, X_test_strat, y_test_strat, fname_test_strat)

    if tflite_metrics_strat:
        P_strat, R_strat, F1_strat, acc_strat, auc_strat, tp_strat, tn_strat, fp_strat, fn_strat, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics_strat
        print("\n📊 TFLITE TEST RESULTS (Random Stratified Test Set):")
        print(f"Precision: {P_strat:.4f}")
        print(f"Recall:    {R_strat:.4f}")
        print(f"F1 score:  {F1_strat:.4f}")
        print(f"Accuracy:  {acc_strat:.4f}")
        print(f"AUC:       {auc_strat:.4f}")
        print(f"TP, TN, FP, FN: {int(tp_strat)}, {int(tn_strat)}, {int(fp_strat)}, {int(fn_strat)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs,
                       "ROC Curve - TFLite Model (Random Stratified Test Set)",
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels,
                              tflite_preds,
                              "Confusion Matrix - TFLite Model",
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        # Compare with original PyTorch test results
        test_results_path = os.path.join(OUTPUT_DIR, "test_results_stratified.csv")
        if os.path.exists(test_results_path):
            strat_pytorch = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results (Random Stratified):")
            print(f"PyTorch → P: {strat_pytorch['Test_Precision']:.4f}, R: {strat_pytorch['Test_Recall']:.4f}, AUC: {strat_pytorch['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P_strat:.4f}, R: {R_strat:.4f}, AUC: {auc_strat:.4f}")

            tol = 1e-3
            p_ok = abs(P_strat - strat_pytorch['Test_Precision']) < tol
            r_ok = abs(R_strat - strat_pytorch['Test_Recall']) < tol
            auc_ok = abs(auc_strat - strat_pytorch['Test_AUC']) < tol

            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")

            # Create metrics comparison plot
            plot_metrics_comparison(strat_pytorch, tflite_metrics_strat,
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed on random stratified test set.")

    print("\n✅ Analysis complete: Random stratified test set evaluated with comprehensive plots and detailed CSV predictions!")
    print(f"✅ Detailed prediction CSVs saved to: {OUTPUT_DIR}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 43 anemic / 361 total

--- Creating Random Stratified Split (Training + Validation + Test) ---
Random stratified train set: 252 anemic / 2172 total
Random stratified val set: 42 anemic / 363 total
Random stratified test set: 42 anemic / 363 total
Original test set: 43 anemic / 361 total

--- Fold 1 ---
Epoch 20/200 Loss: 0.0863
✅ Early stop at epoch 29: P=0.902, R=0.902
Fold 1 → P=0.902, R=0.902

--- Fold 2 ---
Epoch 20/200 Loss: 0.0267
Epoch 40/200 Loss: 0.0227
✅ Early stop at epoch 50: P=0.962, R=0.980
Fold 2 → P=0.962, R=0.980

--- Fold 3 ---
Epoch 20/200 Loss: 0.1168
Epoch 40/200 Loss: 0.0169
✅ Early stop at epoch 56: P=0.980, R=0.980
Fold 3 → P=0.980, R=0.980

--- Fold 4 ---
✅ Early stop at epoch 7: P=0.980, R=0.960
Fold 4 → P=0.980, R=0.960

--- Fold 5 ---
Epoch 20/200 Loss: 3.5117
Epoch 40/200 Loss: 0.9427
Epoch 60/200 Loss: 1.1052
Epoch 80/200 Loss: 0.0062
✅ Early stop at epoch 91: P=0.979, R=0.940
Fold 5 → P=0.979, R=0.940
✅ 

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_stratified_repro/tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_stratified_repro/tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_stratified_repro/tf_model/fingerprint.pb


   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_stratified_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-11-17 08:16:47.987005: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-17 08:16:47.987038: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-17 08:16:47.988750: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_stratified_repro/tf_model
2025-11-17 08:16:48.020428: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-17 08:16:48.020451: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_stratified_repro/tf_model
2025-11-17 08:16:48.031073: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-11-17 08:16:48.053048: I tensorflow/cc/saved_model/loader.cc:215] Run

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_stratified_repro/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on random stratified test set...
🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Random Stratified Test Set):
Precision: 0.8864
Recall:    0.9286
F1 score:  0.9070
Accuracy:  0.9780
AUC:       0.9981
TP, TN, FP, FN: 39, 316, 5, 3
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_stratified_repro/detailed_predictions_tflite.csv

🔍 Comparing with original PyTorch test results (Random Stratified):
PyTorch → P: 0.8864, R: 0.9286, AUC: 0.9981
TFLite  → P: 0.8864, R: 0.9286, AUC: 0.9981
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Random stratified test set evaluated 

In [ ]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added random stratified test split for robustness evaluation
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 780
EPOCHS_CV = 300
BATCH_CV = 14
LR_CV = 0.00003
EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90


BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "Right_EYE_1_eye_stratified_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# CREATE RANDOM STRATIFIED SPLIT (Training + Validation + Test)
# =========================
print("\n--- Creating Random Stratified Split (Training + Validation + Test) ---")
# Combine all data for stratified split
all_imgs = train_imgs + val_imgs + test_imgs
all_lbls = train_lbls + val_lbls + test_lbls
all_filenames = train_filenames + val_filenames + test_filenames

# Create stratified random splits - 75% train, 12.5% val, 12.5% test
X_temp, X_test_strat, y_temp, y_test_strat, fname_temp, fname_test_strat = train_test_split(
    all_imgs, all_lbls, all_filenames,
    test_size=0.125,  # 12.5% for test
    stratify=all_lbls,
    random_state=SEED
)

X_train_strat, X_val_strat, y_train_strat, y_val_strat, fname_train_strat, fname_val_strat = train_test_split(
    X_temp, y_temp, fname_temp,
    test_size=0.142857,  # 12.5% of total (14.2857% of remaining 87.5%) for validation
    stratify=y_temp,
    random_state=SEED
)

print(f"Random stratified train set: {sum(y_train_strat)} anemic / {len(y_train_strat)} total")
print(f"Random stratified val set: {sum(y_val_strat)} anemic / {len(y_val_strat)} total")
print(f"Random stratified test set: {sum(y_test_strat)} anemic / {len(y_test_strat)} total")
print(f"Original test set: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS AND PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []

    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])

    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())

    if len(set(labels_all)) < 2:
        return float("nan"), float("nan"), float("nan"), float("nan"), float("nan"), 0, 0, 0, 0, labels_all, probs, all_filenames, preds

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()
    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc

    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt

    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'],
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'],
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1],
                   tflite_metrics[2], tflite_metrics[3],
                   tflite_metrics[4]]

    x = np.arange(len(metrics))
    width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')

    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    # Use random stratified training data for CV
    X = X_train_strat  # Random stratified training data
    y = y_train_strat
    filenames = fname_train_strat

    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []

    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")

        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]

        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()

            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")

            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR and P <1 and R < 1:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")

        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)

    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]

    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]

    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Random stratified test evaluation
    test_loader_strat = DataLoader(
        SingleEyeDataset(X_test_strat, y_test_strat, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])
    test_metrics_strat = evaluate_with_predictions(model, test_loader_strat, fname_test_strat)

    print("\n📊 FINAL TEST RESULTS (Random Stratified Test Set):")
    print(f"Precision: {test_metrics_strat[0]:.4f}")
    print(f"Recall:    {test_metrics_strat[1]:.4f}")
    print(f"F1 score:  {test_metrics_strat[2]:.4f}")
    print(f"Accuracy:  {test_metrics_strat[3]:.4f}")
    print(f"AUC:       {test_metrics_strat[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics_strat[5])}, {int(test_metrics_strat[6])}, {int(test_metrics_strat[7])}, {int(test_metrics_strat[8])}")


    # Save stratified test results
    _strat_row = [{
         'Test_Precision': test_metrics_strat[0],
         'Test_Recall': test_metrics_strat[1],
         'Test_F1': test_metrics_strat[2],
         'Test_Accuracy': test_metrics_strat[3],
         'Test_AUC': test_metrics_strat[4],
         'Test_TP': test_metrics_strat[5],
         'Test_TN': test_metrics_strat[6],
         'Test_FP': test_metrics_strat[7],
         'Test_FN': test_metrics_strat[8],
         'Best_Fold': best_fold
    }]
    _strat_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_strat_row)[_strat_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results_stratified_780.csv"), index=False)

    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics_strat[11],  # filenames
        test_metrics_strat[9],   # true labels
        test_metrics_strat[12],  # pred labels
        test_metrics_strat[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch_780.csv")
    )

    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics_strat[9], test_metrics_strat[10],
                   "ROC Curve - PyTorch Model (Random Stratified Test Set)",
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics_strat[9],
                          test_metrics_strat[12],
                          "Confusion Matrix - PyTorch Model",
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))

    return model, test_metrics_strat

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)

    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18_780.tflite")

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE)
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")

    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce:
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    model, strat_test_metrics = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")

    # Convert to TFLite
    convert_to_tflite(model, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on random stratified test set
    print("\n🔍 Loading TFLite model and re-evaluating on random stratified test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18_780.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics_strat = evaluate_tflite_on_test_with_predictions(tflite_file, X_test_strat, y_test_strat, fname_test_strat)

    if tflite_metrics_strat:
        P_strat, R_strat, F1_strat, acc_strat, auc_strat, tp_strat, tn_strat, fp_strat, fn_strat, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics_strat
        print("\n📊 TFLITE TEST RESULTS (Random Stratified Test Set):")
        print(f"Precision: {P_strat:.4f}")
        print(f"Recall:    {R_strat:.4f}")
        print(f"F1 score:  {F1_strat:.4f}")
        print(f"Accuracy:  {acc_strat:.4f}")
        print(f"AUC:       {auc_strat:.4f}")
        print(f"TP, TN, FP, FN: {int(tp_strat)}, {int(tn_strat)}, {int(fp_strat)}, {int(fn_strat)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite_780.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs,
                       "ROC Curve - TFLite Model (Random Stratified Test Set)",
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels,
                              tflite_preds,
                              "Confusion Matrix - TFLite Model",
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        # Compare with original PyTorch test results
        test_results_path = os.path.join(OUTPUT_DIR, "test_results_stratified_780.csv")
        if os.path.exists(test_results_path):
            strat_pytorch = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results (Random Stratified):")
            print(f"PyTorch → P: {strat_pytorch['Test_Precision']:.4f}, R: {strat_pytorch['Test_Recall']:.4f}, AUC: {strat_pytorch['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P_strat:.4f}, R: {R_strat:.4f}, AUC: {auc_strat:.4f}")

            tol = 1e-3
            p_ok = abs(P_strat - strat_pytorch['Test_Precision']) < tol
            r_ok = abs(R_strat - strat_pytorch['Test_Recall']) < tol
            auc_ok = abs(auc_strat - strat_pytorch['Test_AUC']) < tol

            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")

            # Create metrics comparison plot
            plot_metrics_comparison(strat_pytorch, tflite_metrics_strat,
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed on random stratified test set.")

    print("\n✅ Analysis complete: Random stratified test set evaluated with comprehensive plots and detailed CSV predictions!")
    print(f"✅ Detailed prediction CSVs saved to: {OUTPUT_DIR}")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 43 anemic / 365 total

--- Creating Random Stratified Split (Training + Validation + Test) ---
Random stratified train set: 254 anemic / 2190 total
Random stratified val set: 42 anemic / 365 total
Random stratified test set: 42 anemic / 365 total
Original test set: 43 anemic / 365 total

--- Fold 1 ---
Epoch 20/300 Loss: 8.4474
Epoch 40/300 Loss: 4.2687
Epoch 60/300 Loss: 1.7011
Epoch 80/300 Loss: 2.0485
Epoch 100/300 Loss: 0.4169
Epoch 120/300 Loss: 0.9647
Epoch 140/300 Loss: 0.2443
Epoch 160/300 Loss: 0.1371
Epoch 180/300 Loss: 6.9355
Epoch 200/300 Loss: 0.2364
Epoch 220/300 Loss: 0.3945
Epoch 240/300 Loss: 1.4637
Epoch 260/300 Loss: 0.2370
Epoch 280/300 Loss: 1.8021
Epoch 300/300 Loss: 0.0343
Fold 1 → P=0.571, R=0.640

--- Fold 2 ---
Epoch 20/300 Loss: 6.5682
Epoch 40/300 Loss: 2.3780
Epoch 60/300 Loss: 1.2212
Epoch 80/300 Loss: 0.3834
Epoch 100/300 Loss: 2.8597
Epoch 120/300 Loss: 0.0816
Epoch 140/300 Loss: 0.1088
Epoch 160/300 Lo

2025-12-02 01:56:10.292621: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-02 01:56:10.294016: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-02 01:56:10.324517: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-02 01:56:10.848138: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



--- Starting TFLite Conversion Pipeline ---
1. Converting PyTorch model to ONNX...
   ✅ ONNX model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro/model.onnx
2. Converting ONNX model to TensorFlow SavedModel...


2025-12-02 01:56:12.678279: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-02 01:56:12.679890: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will 

...

   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-12-02 01:56:17.037359: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-12-02 01:56:17.037393: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-02 01:56:17.039947: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro/tf_model
2025-12-02 01:56:17.071189: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-12-02 01:56:17.071205: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro/tf_model
2025-12-02 01:56:17.103076: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-12-02 01:56:17.103746: I tensorflo

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro/single_eye_resnet18_780.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on random stratified test set...


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Random Stratified Test Set):
Precision: 0.5000
Recall:    0.5000
F1 score:  0.5000
Accuracy:  0.8849
AUC:       0.8952
TP, TN, FP, FN: 21, 302, 21, 21
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro/detailed_predictions_tflite_780.csv

🔍 Comparing with original PyTorch test results (Random Stratified):
PyTorch → P: 0.5000, R: 0.5000, AUC: 0.8952
TFLite  → P: 0.5000, R: 0.5000, AUC: 0.8952
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Random stratified test set evaluated with comprehensive plots and detailed CSV predictions!
✅ Detailed prediction CSVs saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro


In [ ]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added random stratified test split for robustness evaluation
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 1024
EPOCHS_CV = 220
BATCH_CV = 14
LR_CV = 0.00003
EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90


BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "Right_EYE_1_eye_stratified_repro_1024")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# CREATE RANDOM STRATIFIED SPLIT (Training + Validation + Test)
# =========================
print("\n--- Creating Random Stratified Split (Training + Validation + Test) ---")
# Combine all data for stratified split
all_imgs = train_imgs + val_imgs + test_imgs
all_lbls = train_lbls + val_lbls + test_lbls
all_filenames = train_filenames + val_filenames + test_filenames

# Create stratified random splits - 75% train, 12.5% val, 12.5% test
X_temp, X_test_strat, y_temp, y_test_strat, fname_temp, fname_test_strat = train_test_split(
    all_imgs, all_lbls, all_filenames,
    test_size=0.125,  # 12.5% for test
    stratify=all_lbls,
    random_state=SEED
)

X_train_strat, X_val_strat, y_train_strat, y_val_strat, fname_train_strat, fname_val_strat = train_test_split(
    X_temp, y_temp, fname_temp,
    test_size=0.142857,  # 12.5% of total (14.2857% of remaining 87.5%) for validation
    stratify=y_temp,
    random_state=SEED
)

print(f"Random stratified train set: {sum(y_train_strat)} anemic / {len(y_train_strat)} total")
print(f"Random stratified val set: {sum(y_val_strat)} anemic / {len(y_val_strat)} total")
print(f"Random stratified test set: {sum(y_test_strat)} anemic / {len(y_test_strat)} total")
print(f"Original test set: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS AND PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []

    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])

    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())

    if len(set(labels_all)) < 2:
        return float("nan"), float("nan"), float("nan"), float("nan"), float("nan"), 0, 0, 0, 0, labels_all, probs, all_filenames, preds

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()
    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc

    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt

    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'],
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'],
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1],
                   tflite_metrics[2], tflite_metrics[3],
                   tflite_metrics[4]]

    x = np.arange(len(metrics))
    width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')

    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    # Use random stratified training data for CV
    X = X_train_strat  # Random stratified training data
    y = y_train_strat
    filenames = fname_train_strat

    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []

    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")

        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]

        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()

            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")

            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR and P <1 and R < 1:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")

        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)

    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]

    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]

    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Random stratified test evaluation
    test_loader_strat = DataLoader(
        SingleEyeDataset(X_test_strat, y_test_strat, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])
    test_metrics_strat = evaluate_with_predictions(model, test_loader_strat, fname_test_strat)

    print("\n📊 FINAL TEST RESULTS (Random Stratified Test Set):")
    print(f"Precision: {test_metrics_strat[0]:.4f}")
    print(f"Recall:    {test_metrics_strat[1]:.4f}")
    print(f"F1 score:  {test_metrics_strat[2]:.4f}")
    print(f"Accuracy:  {test_metrics_strat[3]:.4f}")
    print(f"AUC:       {test_metrics_strat[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics_strat[5])}, {int(test_metrics_strat[6])}, {int(test_metrics_strat[7])}, {int(test_metrics_strat[8])}")


    # Save stratified test results
    _strat_row = [{
         'Test_Precision': test_metrics_strat[0],
         'Test_Recall': test_metrics_strat[1],
         'Test_F1': test_metrics_strat[2],
         'Test_Accuracy': test_metrics_strat[3],
         'Test_AUC': test_metrics_strat[4],
         'Test_TP': test_metrics_strat[5],
         'Test_TN': test_metrics_strat[6],
         'Test_FP': test_metrics_strat[7],
         'Test_FN': test_metrics_strat[8],
         'Best_Fold': best_fold
    }]
    _strat_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_strat_row)[_strat_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results_stratified.csv"), index=False)

    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics_strat[11],  # filenames
        test_metrics_strat[9],   # true labels
        test_metrics_strat[12],  # pred labels
        test_metrics_strat[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch.csv")
    )

    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics_strat[9], test_metrics_strat[10],
                   "ROC Curve - PyTorch Model (Random Stratified Test Set)",
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics_strat[9],
                          test_metrics_strat[12],
                          "Confusion Matrix - PyTorch Model",
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))

    return model, test_metrics_strat

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)

    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE)
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")

    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce:
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    model, strat_test_metrics = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")

    # Convert to TFLite
    convert_to_tflite(model, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on random stratified test set
    print("\n🔍 Loading TFLite model and re-evaluating on random stratified test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics_strat = evaluate_tflite_on_test_with_predictions(tflite_file, X_test_strat, y_test_strat, fname_test_strat)

    if tflite_metrics_strat:
        P_strat, R_strat, F1_strat, acc_strat, auc_strat, tp_strat, tn_strat, fp_strat, fn_strat, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics_strat
        print("\n📊 TFLITE TEST RESULTS (Random Stratified Test Set):")
        print(f"Precision: {P_strat:.4f}")
        print(f"Recall:    {R_strat:.4f}")
        print(f"F1 score:  {F1_strat:.4f}")
        print(f"Accuracy:  {acc_strat:.4f}")
        print(f"AUC:       {auc_strat:.4f}")
        print(f"TP, TN, FP, FN: {int(tp_strat)}, {int(tn_strat)}, {int(fp_strat)}, {int(fn_strat)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs,
                       "ROC Curve - TFLite Model (Random Stratified Test Set)",
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels,
                              tflite_preds,
                              "Confusion Matrix - TFLite Model",
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        # Compare with original PyTorch test results
        test_results_path = os.path.join(OUTPUT_DIR, "test_results_stratified.csv")
        if os.path.exists(test_results_path):
            strat_pytorch = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results (Random Stratified):")
            print(f"PyTorch → P: {strat_pytorch['Test_Precision']:.4f}, R: {strat_pytorch['Test_Recall']:.4f}, AUC: {strat_pytorch['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P_strat:.4f}, R: {R_strat:.4f}, AUC: {auc_strat:.4f}")

            tol = 1e-3
            p_ok = abs(P_strat - strat_pytorch['Test_Precision']) < tol
            r_ok = abs(R_strat - strat_pytorch['Test_Recall']) < tol
            auc_ok = abs(auc_strat - strat_pytorch['Test_AUC']) < tol

            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")

            # Create metrics comparison plot
            plot_metrics_comparison(strat_pytorch, tflite_metrics_strat,
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed on random stratified test set.")

    print("\n✅ Analysis complete: Random stratified test set evaluated with comprehensive plots and detailed CSV predictions!")
    print(f"✅ Detailed prediction CSVs saved to: {OUTPUT_DIR}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 43 anemic / 365 total

--- Creating Random Stratified Split (Training + Validation + Test) ---
Random stratified train set: 254 anemic / 2190 total
Random stratified val set: 42 anemic / 365 total
Random stratified test set: 42 anemic / 365 total
Original test set: 43 anemic / 365 total

--- Fold 1 ---
Epoch 20/220 Loss: 10.7102
Epoch 40/220 Loss: 2.0890
Epoch 60/220 Loss: 1.9651
Epoch 80/220 Loss: 0.5620
Epoch 100/220 Loss: 1.0330
Epoch 120/220 Loss: 0.2076
Epoch 140/220 Loss: 0.1254
Epoch 160/220 Loss: 0.2101
Epoch 180/220 Loss: 0.4396
Epoch 200/220 Loss: 1.3714
Epoch 220/220 Loss: 0.3631
Fold 1 → P=0.629, R=0.440

--- Fold 2 ---
Epoch 20/220 Loss: 9.6848
Epoch 40/220 Loss: 2.4242
Epoch 60/220 Loss: 1.6010
Epoch 80/220 Loss: 0.3642
Epoch 100/220 Loss: 0.5100
Epoch 120/220 Loss: 0.2886
Epoch 140/220 Loss: 0.4235
Epoch 160/220 Loss: 5.7125
Epoch 180/220 Loss: 0.0698
Epoch 200/220 Loss: 12.0000
Epoch 220/220 Loss: 0.9754
Fold 2 → P=0.6

2025-12-03 00:26:52.736808: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-03 00:26:52.738133: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-03 00:26:52.764866: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-03 00:26:53.312058: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



--- Starting TFLite Conversion Pipeline ---
1. Converting PyTorch model to ONNX...
   ✅ ONNX model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro_1024/model.onnx
2. Converting ONNX model to TensorFlow SavedModel...


2025-12-03 00:26:54.906170: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-03 00:26:54.907932: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will 

INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro_1024/tf_model/assets


...

   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro_1024/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-12-03 00:26:59.349701: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-12-03 00:26:59.349733: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-03 00:26:59.352266: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro_1024/tf_model
2025-12-03 00:26:59.386382: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-12-03 00:26:59.386402: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro_1024/tf_model
2025-12-03 00:26:59.411928: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-12-03 00:26:59.412641: I

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro_1024/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on random stratified test set...


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


🔍 TFLite model input shape: [   1    3 1024 1024]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Random Stratified Test Set):
Precision: 0.6286
Recall:    0.5238
F1 score:  0.5714
Accuracy:  0.9096
AUC:       0.9198
TP, TN, FP, FN: 22, 310, 13, 20
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro_1024/detailed_predictions_tflite.csv

🔍 Comparing with original PyTorch test results (Random Stratified):
PyTorch → P: 0.6286, R: 0.5238, AUC: 0.9198
TFLite  → P: 0.6286, R: 0.5238, AUC: 0.9198
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Random stratified test set evaluated with comprehensive plots and detailed CSV predictions!
✅ Detailed prediction CSVs saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/Right_EYE_1_eye_stratified_repro_1024


In [ ]:
# -*- coding: utf-8 -*-
"""
Reproducible Tri-left-Eye ResNet18 with SHARED BACKBONE & TFLite Verification
----------------------------------------------------------------------------------
✅ Shared ResNet18 across 3 inputs → ~45 MB TFLite
✅ TFLite export with SELECT_TF_OPS (GELU via FlexDelegate)
✅ Re-evaluates .tflite file and compares results with PyTorch
✅ Ensures no silent divergence between frameworks
✅ Added random stratified test split for robustness evaluation
✅ Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
✅ Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison
"""

import os
import json
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold, train_test_split
import warnings

warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 224
EPOCHS_CV = 120
BATCH_CV = 24
LR_CV = 0.00017

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

try:
    cv2.setNumThreads(0)
except Exception:
    pass

os.environ["PYTHONHASHSEED"] = str(SEED)
set_global_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    try:
        print("GPU:", torch.cuda.get_device_name(0))
    except Exception:
        pass

# =========================
# PATHS
# =========================
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
output_dir = os.path.join(base_path, "tri_left_eye_hb_90_repro_bestfold_only_shared_stratified")
os.makedirs(output_dir, exist_ok=True)

def make_full_path(subdirs):
    return {k: os.path.join(base_path, v) for k, v in subdirs.items()}

train_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/'
})
train_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/'
})
val_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/'
})
val_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/'
})
test_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'
})
test_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'
})

# =========================
# UTILS
# =========================
def base_from(fname, suffix):
    return fname[:-len(suffix)] if fname.endswith(suffix) else None

def common_bases_left(dirs_map):
    suffixes = {'left1': '_left_eye_1.png', 'left2': '_left_eye_2.png', 'left3': '_left_eye_3.png'}
    bases_sets = []
    for k in ['left1', 'left2', 'left3']:
        folder = dirs_map[k]
        if not os.path.isdir(folder):
            return []
        names = [f for f in os.listdir(folder) if f.endswith(suffixes[k])]
        bases = {base_from(f, suffixes[k]) for f in names if base_from(f, suffixes[k]) is not None}
        bases_sets.append(bases)
    if not bases_sets:
        return []
    inter = set.intersection(*bases_sets)
    return sorted(inter)

def load_tri_images_by_bases_with_filenames(dirs_map, bases):
    out = {'r1': [], 'r2': [], 'r3': [], 'filenames': []}
    key_map = {'r1': ('left1', '_left_eye_1.png'), 'r2': ('left2', '_left_eye_2.png'), 'r3': ('left3', '_left_eye_3.png')}
    for b in bases:
        imgs, failed = {}, False
        for short_k, (long_k, suf) in key_map.items():
            path = os.path.join(dirs_map[long_k], b + suf)
            if not os.path.isfile(path):
                failed = True
                break
            img = cv2.imread(path)
            if img is None:
                failed = True
                break
            imgs[short_k] = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if not failed:
            out['r1'].append(imgs['r1'])
            out['r2'].append(imgs['r2'])
            out['r3'].append(imgs['r3'])
            out['filenames'].append(b)  # Use base name as identifier
    return out

def prepare_dataset_left_with_filenames(anemic_dirs, non_dirs, split_name="(split)"):
    bases_a = common_bases_left(anemic_dirs)
    bases_n = common_bases_left(non_dirs)
    imgs_a = load_tri_images_by_bases_with_filenames(anemic_dirs, bases_a)
    imgs_n = load_tri_images_by_bases_with_filenames(non_dirs, bases_n)

    # Combine anemic and non-anemic data
    data = {
        'r1': imgs_a['r1'] + imgs_n['r1'],
        'r2': imgs_a['r2'] + imgs_n['r2'],
        'r3': imgs_a['r3'] + imgs_n['r3'],
        'filenames': imgs_a['filenames'] + imgs_n['filenames'],
        'label': [1]*len(imgs_a['r1']) + [0]*len(imgs_n['r1'])
    }

    print(f"✅ {split_name}: anemic={len(imgs_a['r1'])}, non-anemic={len(imgs_n['r1'])}, total={len(data['label'])}")
    try:
        df_bases = pd.DataFrame({'class': ['anemic']*len(bases_a) + ['non_anemic']*len(bases_n),
                                 'base_id': bases_a + bases_n})
        df_bases.to_csv(os.path.join(output_dir, f"{split_name.lower()}_used_base_ids_left.csv"), index=False)
    except Exception:
        pass
    return data

def count_files(d):
    return sum(1 for f in sorted(os.listdir(d)) if f.endswith(".png")) if os.path.isdir(d) else 0

def print_dir_stats(title, dirs_map):
    print(f"\n📂 {title}")
    for k in ['left1','left2','left3']:
        p = dirs_map[k]; c = count_files(p)
        print(f"{k:7s} | {p} | files={c}")

# =========================
# LOAD DATA
# =========================

print_dir_stats("TEST  anemic (LEFT)", test_dirs_anemic)
print_dir_stats("TEST  non-anemic (LEFT)", test_dirs_non)

train_data = prepare_dataset_left_with_filenames(train_dirs_anemic, train_dirs_non, split_name="TRAIN")
val_data   = prepare_dataset_left_with_filenames(val_dirs_anemic,   val_dirs_non,   split_name="VAL")
test_data  = prepare_dataset_left_with_filenames(test_dirs_anemic,  test_dirs_non,  split_name="TEST")

if len(train_data['label']) == 0:
    raise RuntimeError("No tri-left-eye TRAIN samples found.")

# =========================
# CREATE RANDOM STRATIFIED SPLIT (Training + Validation + Test)
# =========================
print("\n--- Creating Random Stratified Split (Training + Validation + Test) ---")
# Combine all data for stratified split
all_r1 = train_data['r1'] + val_data['r1'] + test_data['r1']
all_r2 = train_data['r2'] + val_data['r2'] + test_data['r2']
all_r3 = train_data['r3'] + val_data['r3'] + test_data['r3']
all_filenames = train_data['filenames'] + val_data['filenames'] + test_data['filenames']
all_labels = train_data['label'] + val_data['label'] + test_data['label']

# Create stratified random splits - 75% train, 12.5% val, 12.5% test
X_temp_r1, X_test_strat_r1, X_temp_r2, X_test_strat_r2, X_temp_r3, X_test_strat_r3, fname_temp, fname_test_strat, y_temp, y_test_strat = train_test_split(
    all_r1, all_r2, all_r3, all_filenames, all_labels,
    test_size=0.125,  # 12.5% for test
    stratify=all_labels,
    random_state=SEED
)

X_train_strat_r1, X_val_strat_r1, X_train_strat_r2, X_val_strat_r2, X_train_strat_r3, X_val_strat_r3, fname_train_strat, fname_val_strat, y_train_strat, y_val_strat = train_test_split(
    X_temp_r1, X_temp_r2, X_temp_r3, fname_temp, y_temp,
    test_size=0.142857,  # 12.5% of total (14.2857% of remaining 87.5%) for validation
    stratify=y_temp,
    random_state=SEED
)

strat_train_data = {
    'r1': X_train_strat_r1,
    'r2': X_train_strat_r2,
    'r3': X_train_strat_r3,
    'filenames': fname_train_strat,
    'label': y_train_strat
}

strat_val_data = {
    'r1': X_val_strat_r1,
    'r2': X_val_strat_r2,
    'r3': X_val_strat_r3,
    'filenames': fname_val_strat,
    'label': y_val_strat
}

strat_test_data = {
    'r1': X_test_strat_r1,
    'r2': X_test_strat_r2,
    'r3': X_test_strat_r3,
    'filenames': fname_test_strat,
    'label': y_test_strat
}

print(f"Random stratified train set: {sum(y_train_strat)} anemic / {len(y_train_strat)} total")
print(f"Random stratified val set: {sum(y_val_strat)} anemic / {len(y_val_strat)} total")
print(f"Random stratified test set: {sum(y_test_strat)} anemic / {len(y_test_strat)} total")
print(f"Original test set: {sum(test_data['label'])} anemic / {len(test_data['label'])} total")

# =========================
# DATASET
# =========================
class TrileftDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
    def __len__(self):
        return len(self.data['label'])
    def __getitem__(self, idx):
        images = [self.data[k][idx] for k in ['r1','r2','r3']]
        images = [self.transform(img) for img in images]
        label = self.data['label'][idx]
        return images, label

def seed_worker(worker_id):
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    torch.manual_seed(worker_seed)

def make_loader(dataset, batch_size, shuffle):
    g = torch.Generator()
    g.manual_seed(SEED)
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY and (device.type=='cuda'),
        worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
        generator=g,
    )

# =========================
# MODEL: SHARED RESNET18 BACKBONE
# =========================
class TriResNetleft(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Identity()

        self.fusion = nn.Sequential(
            nn.Linear(3 * 512, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(0.4),
            nn.Linear(512, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(0.25),
            nn.Linear(128, 1)
        )

    def forward(self, x1, x2, x3):
        f1 = self.backbone(x1)
        f2 = self.backbone(x2)
        f3 = self.backbone(x3)
        x = torch.cat([f1, f2, f3], dim=1)
        return self.fusion(x)

# =========================
# EVALUATION (PyTorch) WITH PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []
    all_filenames = []

    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])

    for imgs, labels in loader:
        x1, x2, x3 = [img.to(device).float() for img in imgs]
        labels = labels.to(device).float().unsqueeze(1)
        out = model(x1, x2, x3)
        prob = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (prob > 0.5).astype(int)
        all_preds.extend(pred.tolist())
        all_probs.extend(prob.tolist())
        all_labels.extend(labels.cpu().numpy().flatten().tolist())

    if len(all_labels) == 0 or len(set(all_labels)) < 2:
        return [float('nan')] * 9 + [all_labels, all_probs, all_filenames, all_preds]

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, all_filenames, all_preds

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(best_model: nn.Module, output_dir: str, resolution: int, tflite_filename: str):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    best_model.eval().to('cpu')
    dummy_inputs = tuple(torch.randn(1, 3, resolution, resolution) for _ in range(3))
    onnx_path = os.path.join(output_dir, "tri_left_model.onnx")
    tf_path = os.path.join(output_dir, "tri_left_tf_model")
    tflite_path = os.path.join(output_dir, tflite_filename)

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # Step 1: PyTorch -> ONNX
    print("1. Converting to ONNX...")
    try:
        torch.onnx.export(
            best_model,
            dummy_inputs,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input1', 'input2', 'input3'],
            output_names=['output']
        )
        print(f"   ✅ ONNX saved: {onnx_path}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

    # Step 2: ONNX -> TensorFlow
    print("2. ONNX -> TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        if os.path.exists(tf_path):
            import shutil
            shutil.rmtree(tf_path)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TF SavedModel saved: {tf_path}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

    # Step 3: TF -> TFLite with SELECT_TF_OPS
    print("3. TF -> TFLite (SELECT_TF_OPS)...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        tflite_model = converter.convert()
        with open(tflite_path, "wb") as f:
            f.write(tflite_model)
        size_mb = os.path.getsize(tflite_path) / (1024 * 1024)
        print(f"   ✅ TFLite saved: {tflite_path} ({size_mb:.2f} MB)")
        return tflite_path
    except Exception as e:
        print(f"❌ Conversion failed: {e}")
        return None

# =========================
# TFLITE RE-EVALUATION WITH PREDICTIONS
# =========================
def evaluate_tflite_model_with_predictions(tflite_path, test_data, resolution):
    import tensorflow as tf
    from PIL import Image
    import numpy as np
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    sorted_inputs = sorted(input_details, key=lambda x: x['name'])
    print(f"🔍 TFLite input names: {[d['name'] for d in sorted_inputs]}")

    first_shape = sorted_inputs[0]['shape']
    if len(first_shape) == 4:
        if first_shape[1] == 3:  # [B,C,H,W]
            layout = 'NCHW'
        else:  # [B,H,W,C]
            layout = 'NHWC'

    resize_h, resize_w = int(first_shape[1] if layout == 'NHWC' else first_shape[2]), \
                         int(first_shape[2] if layout == 'NHWC' else first_shape[3])

    print(f"   ➤ Detected layout: {layout}, size: {resize_h}x{resize_w}")

    def preprocess_pil_style(img_rgb):
        img_pil = Image.fromarray(img_rgb)
        img_resized = img_pil.resize((resize_w, resize_h), Image.BILINEAR)
        tensor = to_tensor(img_resized)
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()

    all_preds, all_probs, all_labels, all_filenames = [], [], [], test_data['filenames']

    for i in range(len(test_data['label'])):
        imgs = [test_data['r1'][i], test_data['r2'][i], test_data['r3'][i]]
        label = test_data['label'][i]

        for idx, detail in enumerate(sorted_inputs):
            raw_img = imgs[idx]
            processed = preprocess_pil_style(raw_img)

            if layout == 'NCHW':
                model_input = np.expand_dims(processed, axis=0).astype(detail['dtype'])
            else:
                nhwc = np.transpose(processed, (1, 2, 0))
                model_input = np.expand_dims(nhwc, axis=0).astype(detail['dtype'])

            interpreter.set_tensor(detail['index'], model_input)

        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])
        logit = float(np.array(output).reshape(-1)[0])
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        all_preds.append(pred)
        all_probs.append(prob)
        all_labels.append(label)

    if len(set(all_labels)) < 2:
        auc = float('nan')
    else:
        auc = roc_auc_score(all_labels, all_probs)

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()

    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, all_filenames, all_preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc

    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt

    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'],
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'],
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1],
                   tflite_metrics[2], tflite_metrics[3],
                   tflite_metrics[4]]

    x = np.arange(len(metrics))
    width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')

    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# MAIN TRAINING LOOP
# =========================
if __name__ == "__main__":
    resolution = RESOLUTION
    results = []
    cv_index_records = []

    print(f"\n===== Processing LEFT resolution: {resolution} =====")
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    labels_np = np.array(strat_train_data['label'])

    fold = 1
    for train_idx, val_idx in kf.split(np.zeros_like(labels_np), labels_np):
        print(f"\n--- LEFT Fold {fold} ---")
        cv_index_records.append({"fold": fold, "train_indices": train_idx.tolist(), "val_indices": val_idx.tolist()})

        train_subset = {k: [v[i] for i in train_idx] for k, v in strat_train_data.items()}
        val_subset   = {k: [v[i] for i in val_idx]   for k, v in strat_train_data.items()}

        train_loader = make_loader(TrileftDataset(train_subset, train_transform), BATCH_CV, True)
        val_loader   = make_loader(TrileftDataset(val_subset,   test_transform),  BATCH_CV, False)

        model = TriResNetleft().to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=LR_CV)
        scaler = GradScaler(enabled=(USE_AMP and device.type == "cuda"))

        for epoch in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in train_loader:
                x1, x2, x3 = [img.to(device).float() for img in imgs]
                labels = labels.to(device).float().unsqueeze(1)
                optimizer.zero_grad(set_to_none=True)
                with autocast(enabled=(USE_AMP and device.type == "cuda")):
                    out = model(x1, x2, x3)
                    loss = criterion(out, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_loss += loss.item()
            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{EPOCHS_CV}] Loss: {total_loss:.6f}")

        val_metrics = evaluate_with_predictions(model, val_loader, val_subset['filenames'])
        result_row = {
            'EyeSet': 'LEFT',
            'Resolution': resolution,
            'Fold': fold,
            'Val_Precision': val_metrics[0],
            'Val_Recall': val_metrics[1],
            'Val_F1': val_metrics[2],
            'Val_Accuracy': val_metrics[3],
            'Val_AUC': val_metrics[4],
            'Val_TP': val_metrics[5],
            'Val_TN': val_metrics[6],
            'Val_FP': val_metrics[7],
            'Val_FN': val_metrics[8]
        }
        results.append(result_row)
        print(results)
        if SAVE_EVERY_FOLD_MODEL:
            fold_path = os.path.join(output_dir, f"left_cv_fold_{fold}_res{resolution}.pt")
            torch.save({'model_state': model.state_dict()}, fold_path)

        fold += 1

    # Save CV results
    pd.DataFrame(results).to_csv(os.path.join(output_dir, "left_val_cross_validation_results.csv"), index=False)
    with open(os.path.join(output_dir, "left_cv_indices.json"), "w") as f:
        json.dump(cv_index_records, f, indent=2)

    # Select best fold
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision','Val_Recall']].min(axis=1)
    candidates = df[(df['Val_Precision'] >= 0.90) & (df['Val_Recall'] >= 0.90)]
    best = candidates.sort_values(['Val_F1'], ascending=False).iloc[0] if len(candidates) > 0 else \
           df.sort_values(['minPR','Val_F1'], ascending=False).iloc[0]
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold}")

    # Load best model
    ckpt_path = os.path.join(output_dir, f"left_cv_fold_{best_fold}_res{resolution}.pt")
    best_model = TriResNetleft().to(device)
    state = torch.load(ckpt_path, map_location=device)
    best_model.load_state_dict(state['model_state'])

    # Test evaluation (PyTorch) - Random Stratified Test Set
    test_loader_strat = make_loader(TrileftDataset(strat_test_data, test_transform), BATCH_CV, False)
    test_metrics_strat = evaluate_with_predictions(best_model, test_loader_strat, strat_test_data['filenames'])

    test_results_df = pd.DataFrame([{
        'ChosenFold': best_fold,
        'Test_Precision': test_metrics_strat[0],
        'Test_Recall': test_metrics_strat[1],
        'Test_F1': test_metrics_strat[2],
        'Test_Accuracy': test_metrics_strat[3],
        'Test_AUC': test_metrics_strat[4],
        'Test_TP': test_metrics_strat[5],
        'Test_TN': test_metrics_strat[6],
        'Test_FP': test_metrics_strat[7],
        'Test_FN': test_metrics_strat[8],
        'Test_Type': 'Random_Stratified'
    }])
    test_results_df.to_csv(os.path.join(output_dir, "left_bestfold_test_results_stratified.csv"), index=False)

    print("\n📊 TEST Results (Random Stratified Test Set - Shared Backbone):")
    print(test_results_df.to_string(index=False))

    # Save detailed PyTorch predictions to CSV
    save_predictions_to_csv(
        test_metrics_strat[11],  # filenames
        test_metrics_strat[9],   # true labels
        test_metrics_strat[12],  # pred labels
        test_metrics_strat[10],  # pred probs
        os.path.join(output_dir, "detailed_predictions_pytorch.csv")
    )

    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics_strat[9], test_metrics_strat[10],
                   "ROC Curve - PyTorch Model (Random Stratified Test Set)",
                   os.path.join(output_dir, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics_strat[9],
                          test_metrics_strat[12],
                          "Confusion Matrix - PyTorch Model",
                          os.path.join(output_dir, "confusion_matrix_pytorch.png"))

    # Convert to TFLite
    tflite_filename = "tri_left_eye_resnet18_shared_stratified.tflite"
    tflite_path = convert_to_tflite(best_model.cpu(), output_dir, resolution, tflite_filename)

    if tflite_path:
        size_mb = os.path.getsize(tflite_path) / (1024 * 1024)
        print(f"\n🎉 SUCCESS! Final TFLite model size: {size_mb:.2f} MB")
        print(f"📍 Path: {tflite_path}")

        # --- 🔍 Re-evaluate TFLite model on random stratified test set ---
        print("\n🔍 Re-evaluating TFLite model on random stratified test set...")
        try:
            tflite_metrics = evaluate_tflite_model_with_predictions(tflite_path, strat_test_data, resolution)
            tflite_results_df = pd.DataFrame([{
                'Source': 'TFLite',
                'Test_Precision': tflite_metrics[0],
                'Test_Recall': tflite_metrics[1],
                'Test_F1': tflite_metrics[2],
                'Test_Accuracy': tflite_metrics[3],
                'Test_AUC': tflite_metrics[4],
                'Test_TP': tflite_metrics[5],
                'Test_TN': tflite_metrics[6],
                'Test_FP': tflite_metrics[7],
                'Test_FN': tflite_metrics[8],
                'Test_Type': 'Random_Stratified'
            }])

            combined = pd.concat([
                test_results_df.assign(Source='PyTorch'),
                tflite_results_df
            ], ignore_index=True)

            print("\n📊 COMPARISON: PyTorch vs TFLite (Random Stratified Test Set)")
            print(combined.to_string(index=False))
            combined.to_csv(os.path.join(output_dir, "pytorch_vs_tflite_comparison_stratified.csv"), index=False)

            # Save detailed TFLite predictions to CSV
            save_predictions_to_csv(
                tflite_metrics[11],  # filenames
                tflite_metrics[9],   # true labels
                tflite_metrics[12],  # pred labels
                tflite_metrics[10],  # pred probs
                os.path.join(output_dir, "detailed_predictions_tflite.csv")
            )

            # Plot ROC curve and confusion matrix for TFLite model
            plot_roc_curve(tflite_metrics[9], tflite_metrics[10],
                           "ROC Curve - TFLite Model (Random Stratified Test Set)",
                           os.path.join(output_dir, "roc_curve_tflite.png"))
            plot_confusion_matrix(tflite_metrics[9],
                                  tflite_metrics[12],
                                  "Confusion Matrix - TFLite Model",
                                  os.path.join(output_dir, "confusion_matrix_tflite.png"))

            # Create metrics comparison plot
            plot_metrics_comparison(test_results_df.iloc[0].to_dict(), tflite_metrics,
                                   os.path.join(output_dir, "metrics_comparison.png"))

            tol = 1e-3
            if (abs(tflite_metrics[2] - test_metrics_strat[2]) < tol and
                abs(tflite_metrics[4] - test_metrics_strat[4]) < tol):
                print("✅ TFLite results MATCH PyTorch within tolerance.")
            else:
                print("⚠️ WARNING: TFLite results differ significantly from PyTorch!")
        except Exception as e:
            print(f"❌ TFLite evaluation failed: {e}")
    else:
        print("❌ TFLite conversion failed.")

    print("\n✅ Pipeline completed. Model evaluated on random stratified test set with shared backbone.")
    print(f"✅ Detailed prediction CSVs and plots saved to: {output_dir}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3

📂 TEST  anemic (LEFT)
left1   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=44
left2   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=42
left3   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=43

📂 TEST  non-anemic (LEFT)
left1   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/ | files=314
left2   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_r

INFO:absl:Function `__call__` contains input name(s) input, x, y with unsupported characters which will be renamed to onnx_tf_prefix_identity_49_input, transpose_187_x, onnx_tf_prefix__fusion_fusion_1_add_1_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared_stratified/tri_left_tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared_stratified/tri_left_tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared_stratified/tri_left_tf_model/fingerprint.pb


   ✅ TF SavedModel saved: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared_stratified/tri_left_tf_model
3. TF -> TFLite (SELECT_TF_OPS)...


2025-11-18 05:40:36.678763: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-18 05:40:36.678806: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-18 05:40:36.683671: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared_stratified/tri_left_tf_model
2025-11-18 05:40:36.724432: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-18 05:40:36.724453: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared_stratified/tri_left_tf_model
2025-11-18 05:40:36.758856: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-1

   ✅ TFLite saved: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared_stratified/tri_left_eye_resnet18_shared_stratified.tflite (45.92 MB)

🎉 SUCCESS! Final TFLite model size: 45.92 MB
📍 Path: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared_stratified/tri_left_eye_resnet18_shared_stratified.tflite

🔍 Re-evaluating TFLite model on random stratified test set...


INFO: Created TensorFlow Lite delegate for select TF ops.
2025-11-18 05:40:38.236075: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-18 05:40:38.237343: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO: TfLiteFlexDelegate delegate: 2 nodes delegated out of 285 nodes with 2 partitions.



🔍 TFLite input names: ['serving_default_input1:0', 'serving_default_input2:0', 'serving_default_input3:0']
   ➤ Detected layout: NCHW, size: 224x224

📊 COMPARISON: PyTorch vs TFLite (Random Stratified Test Set)
 ChosenFold  Test_Precision  Test_Recall  Test_F1  Test_Accuracy  Test_AUC  Test_TP  Test_TN  Test_FP  Test_FN         Test_Type  Source
        5.0           0.875        0.875    0.875       0.970845  0.994554       35      298        5        5 Random_Stratified PyTorch
        NaN           0.875        0.875    0.875       0.970845  0.994554       35      298        5        5 Random_Stratified  TFLite
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared_stratified/detailed_predictions_tflite.csv
✅ TFLite results MATCH PyTorch within tolerance.

✅ Pipeline completed. Model evaluated on random stratified test set with shared backbone.
✅ Detailed prediction CSVs and plots saved t

In [ ]:
# -*- coding: utf-8 -*-
"""
Reproducible Tri-right-Eye ResNet18 with SHARED BACKBONE & TFLite Verification
----------------------------------------------------------------------------------
✅ Shared ResNet18 across 3 inputs → ~45 MB TFLite
✅ TFLite export with SELECT_TF_OPS (GELU via FlexDelegate)
✅ Re-evaluates .tflite file and compares results with PyTorch
✅ Ensures no silent divergence between frameworks
✅ Added random stratified test split for robustness evaluation
✅ Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
✅ Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison
"""

import os
import json
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold, train_test_split
import warnings

warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 224  # Only one resolution used now
EPOCHS_CV = 150
BATCH_CV = 28
LR_CV = 0.00022
# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

try:
    cv2.setNumThreads(0)
except Exception:
    pass

os.environ["PYTHONHASHSEED"] = str(SEED)
set_global_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    try:
        print("GPU:", torch.cuda.get_device_name(0))
    except Exception:
        pass

# =========================
# PATHS
# =========================
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
output_dir = os.path.join(base_path, "tri_right_eye_hb_90_repro_bestfold_only_shared_stratified")
os.makedirs(output_dir, exist_ok=True)

def make_full_path(subdirs):
    return {k: os.path.join(base_path, v) for k, v in subdirs.items()}

train_dirs_anemic = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/'
})
train_dirs_non = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/'
})
val_dirs_anemic = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/'
})
val_dirs_non = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/'
})
test_dirs_anemic = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'
})
test_dirs_non = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'
})

# =========================
# UTILS
# =========================
def base_from(fname, suffix):
    return fname[:-len(suffix)] if fname.endswith(suffix) else None

def common_bases_right(dirs_map):
    suffixes = {'right1': '_right_eye_1.png', 'right2': '_right_eye_2.png', 'right3': '_right_eye_3.png'}
    bases_sets = []
    for k in ['right1', 'right2', 'right3']:
        folder = dirs_map[k]
        if not os.path.isdir(folder):
            return []
        names = [f for f in os.listdir(folder) if f.endswith(suffixes[k])]
        bases = {base_from(f, suffixes[k]) for f in names if base_from(f, suffixes[k]) is not None}
        bases_sets.append(bases)
    if not bases_sets:
        return []
    inter = set.intersection(*bases_sets)
    return sorted(inter)

def load_tri_images_by_bases_with_filenames(dirs_map, bases):
    out = {'r1': [], 'r2': [], 'r3': [], 'filenames': []}
    key_map = {'r1': ('right1', '_right_eye_1.png'), 'r2': ('right2', '_right_eye_2.png'), 'r3': ('right3', '_right_eye_3.png')}
    for b in bases:
        imgs, failed = {}, False
        for short_k, (long_k, suf) in key_map.items():
            path = os.path.join(dirs_map[long_k], b + suf)
            if not os.path.isfile(path):
                failed = True
                break
            img = cv2.imread(path)
            if img is None:
                failed = True
                break
            imgs[short_k] = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if not failed:
            out['r1'].append(imgs['r1'])
            out['r2'].append(imgs['r2'])
            out['r3'].append(imgs['r3'])
            out['filenames'].append(b)  # Use base name as identifier
    return out

def prepare_dataset_right_with_filenames(anemic_dirs, non_dirs, split_name="(split)"):
    bases_a = common_bases_right(anemic_dirs)
    bases_n = common_bases_right(non_dirs)
    imgs_a = load_tri_images_by_bases_with_filenames(anemic_dirs, bases_a)
    imgs_n = load_tri_images_by_bases_with_filenames(non_dirs, bases_n)

    # Combine anemic and non-anemic data
    data = {
        'r1': imgs_a['r1'] + imgs_n['r1'],
        'r2': imgs_a['r2'] + imgs_n['r2'],
        'r3': imgs_a['r3'] + imgs_n['r3'],
        'filenames': imgs_a['filenames'] + imgs_n['filenames'],
        'label': [1]*len(imgs_a['r1']) + [0]*len(imgs_n['r1'])
    }

    print(f"✅ {split_name}: anemic={len(imgs_a['r1'])}, non-anemic={len(imgs_n['r1'])}, total={len(data['label'])}")
    try:
        df_bases = pd.DataFrame({'class': ['anemic']*len(bases_a) + ['non_anemic']*len(bases_n),
                                 'base_id': bases_a + bases_n})
        df_bases.to_csv(os.path.join(output_dir, f"{split_name.lower()}_used_base_ids_right.csv"), index=False)
    except Exception:
        pass
    return data

def count_files(d):
    return sum(1 for f in sorted(os.listdir(d)) if f.endswith(".png")) if os.path.isdir(d) else 0

def print_dir_stats(title, dirs_map):
    print(f"\n📂 {title}")
    for k in ['right1','right2','right3']:
        p = dirs_map[k]; c = count_files(p)
        print(f"{k:7s} | {p} | files={c}")

# =========================
# LOAD DATA
# =========================

print_dir_stats("TEST  anemic (right)", test_dirs_anemic)
print_dir_stats("TEST  non-anemic (right)", test_dirs_non)

train_data = prepare_dataset_right_with_filenames(train_dirs_anemic, train_dirs_non, split_name="TRAIN")
val_data   = prepare_dataset_right_with_filenames(val_dirs_anemic,   val_dirs_non,   split_name="VAL")
test_data  = prepare_dataset_right_with_filenames(test_dirs_anemic,  test_dirs_non,  split_name="TEST")

if len(train_data['label']) == 0:
    raise RuntimeError("No tri-right-eye TRAIN samples found.")

# =========================
# CREATE RANDOM STRATIFIED SPLIT (Training + Validation + Test)
# =========================
print("\n--- Creating Random Stratified Split (Training + Validation + Test) ---")
# Combine all data for stratified split
all_r1 = train_data['r1'] + val_data['r1'] + test_data['r1']
all_r2 = train_data['r2'] + val_data['r2'] + test_data['r2']
all_r3 = train_data['r3'] + val_data['r3'] + test_data['r3']
all_filenames = train_data['filenames'] + val_data['filenames'] + test_data['filenames']
all_labels = train_data['label'] + val_data['label'] + test_data['label']

# Create stratified random splits - 75% train, 12.5% val, 12.5% test
X_temp_r1, X_test_strat_r1, X_temp_r2, X_test_strat_r2, X_temp_r3, X_test_strat_r3, fname_temp, fname_test_strat, y_temp, y_test_strat = train_test_split(
    all_r1, all_r2, all_r3, all_filenames, all_labels,
    test_size=0.125,  # 12.5% for test
    stratify=all_labels,
    random_state=SEED
)

X_train_strat_r1, X_val_strat_r1, X_train_strat_r2, X_val_strat_r2, X_train_strat_r3, X_val_strat_r3, fname_train_strat, fname_val_strat, y_train_strat, y_val_strat = train_test_split(
    X_temp_r1, X_temp_r2, X_temp_r3, fname_temp, y_temp,
    test_size=0.142857,  # 12.5% of total (14.2857% of remaining 87.5%) for validation
    stratify=y_temp,
    random_state=SEED
)

strat_train_data = {
    'r1': X_train_strat_r1,
    'r2': X_train_strat_r2,
    'r3': X_train_strat_r3,
    'filenames': fname_train_strat,
    'label': y_train_strat
}

strat_val_data = {
    'r1': X_val_strat_r1,
    'r2': X_val_strat_r2,
    'r3': X_val_strat_r3,
    'filenames': fname_val_strat,
    'label': y_val_strat
}

strat_test_data = {
    'r1': X_test_strat_r1,
    'r2': X_test_strat_r2,
    'r3': X_test_strat_r3,
    'filenames': fname_test_strat,
    'label': y_test_strat
}

print(f"Random stratified train set: {sum(y_train_strat)} anemic / {len(y_train_strat)} total")
print(f"Random stratified val set: {sum(y_val_strat)} anemic / {len(y_val_strat)} total")
print(f"Random stratified test set: {sum(y_test_strat)} anemic / {len(y_test_strat)} total")
print(f"Original test set: {sum(test_data['label'])} anemic / {len(test_data['label'])} total")

# =========================
# DATASET
# =========================
class TrirightDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
    def __len__(self):
        return len(self.data['label'])
    def __getitem__(self, idx):
        images = [self.data[k][idx] for k in ['r1','r2','r3']]
        images = [self.transform(img) for img in images]
        label = self.data['label'][idx]
        return images, label

def seed_worker(worker_id):
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    torch.manual_seed(worker_seed)

def make_loader(dataset, batch_size, shuffle):
    g = torch.Generator()
    g.manual_seed(SEED)
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY and (device.type=='cuda'),
        worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
        generator=g,
    )

# =========================
# MODEL: SHARED RESNET18 BACKBONE
# =========================
class TriResNetright(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Identity()

        self.fusion = nn.Sequential(
            nn.Linear(3 * 512, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(0.4),
            nn.Linear(512, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(0.25),
            nn.Linear(128, 1)
        )

    def forward(self, x1, x2, x3):
        f1 = self.backbone(x1)
        f2 = self.backbone(x2)
        f3 = self.backbone(x3)
        x = torch.cat([f1, f2, f3], dim=1)
        return self.fusion(x)

# =========================
# EVALUATION (PyTorch) WITH PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []
    all_filenames = []

    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])

    for imgs, labels in loader:
        x1, x2, x3 = [img.to(device).float() for img in imgs]
        labels = labels.to(device).float().unsqueeze(1)
        out = model(x1, x2, x3)
        prob = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (prob > 0.5).astype(int)
        all_preds.extend(pred.tolist())
        all_probs.extend(prob.tolist())
        all_labels.extend(labels.cpu().numpy().flatten().tolist())

    if len(all_labels) == 0 or len(set(all_labels)) < 2:
        return [float('nan')] * 9 + [all_labels, all_probs, all_filenames, all_preds]

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, all_filenames, all_preds

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(best_model: nn.Module, output_dir: str, resolution: int, tflite_filename: str):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    best_model.eval().to('cpu')
    dummy_inputs = tuple(torch.randn(1, 3, resolution, resolution) for _ in range(3))
    onnx_path = os.path.join(output_dir, "tri_right_model.onnx")
    tf_path = os.path.join(output_dir, "tri_right_tf_model")
    tflite_path = os.path.join(output_dir, tflite_filename)

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # Step 1: PyTorch -> ONNX
    print("1. Converting to ONNX...")
    try:
        torch.onnx.export(
            best_model,
            dummy_inputs,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input1', 'input2', 'input3'],
            output_names=['output']
        )
        print(f"   ✅ ONNX saved: {onnx_path}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

    # Step 2: ONNX -> TensorFlow
    print("2. ONNX -> TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        if os.path.exists(tf_path):
            import shutil
            shutil.rmtree(tf_path)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TF SavedModel saved: {tf_path}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

    # Step 3: TF -> TFLite with SELECT_TF_OPS
    print("3. TF -> TFLite (SELECT_TF_OPS)...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        tflite_model = converter.convert()
        with open(tflite_path, "wb") as f:
            f.write(tflite_model)
        size_mb = os.path.getsize(tflite_path) / (1024 * 1024)
        print(f"   ✅ TFLite saved: {tflite_path} ({size_mb:.2f} MB)")
        return tflite_path
    except Exception as e:
        print(f"❌ Conversion failed: {e}")
        return None

# =========================
# TFLITE RE-EVALUATION WITH PREDICTIONS
# =========================
def evaluate_tflite_model_with_predictions(tflite_path, test_data, resolution):
    import tensorflow as tf
    from PIL import Image
    import numpy as np
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    sorted_inputs = sorted(input_details, key=lambda x: x['name'])
    print(f"🔍 TFLite input names: {[d['name'] for d in sorted_inputs]}")

    first_shape = sorted_inputs[0]['shape']
    if len(first_shape) == 4:
        if first_shape[1] == 3:  # [B,C,H,W]
            layout = 'NCHW'
        else:  # [B,H,W,C]
            layout = 'NHWC'

    resize_h, resize_w = int(first_shape[1] if layout == 'NHWC' else first_shape[2]), \
                         int(first_shape[2] if layout == 'NHWC' else first_shape[3])

    print(f"   ➤ Detected layout: {layout}, size: {resize_h}x{resize_w}")

    def preprocess_pil_style(img_rgb):
        img_pil = Image.fromarray(img_rgb)
        img_resized = img_pil.resize((resize_w, resize_h), Image.BILINEAR)
        tensor = to_tensor(img_resized)
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()

    all_preds, all_probs, all_labels, all_filenames = [], [], [], test_data['filenames']

    for i in range(len(test_data['label'])):
        imgs = [test_data['r1'][i], test_data['r2'][i], test_data['r3'][i]]
        label = test_data['label'][i]

        for idx, detail in enumerate(sorted_inputs):
            raw_img = imgs[idx]
            processed = preprocess_pil_style(raw_img)

            if layout == 'NCHW':
                model_input = np.expand_dims(processed, axis=0).astype(detail['dtype'])
            else:
                nhwc = np.transpose(processed, (1, 2, 0))
                model_input = np.expand_dims(nhwc, axis=0).astype(detail['dtype'])

            interpreter.set_tensor(detail['index'], model_input)

        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])
        logit = float(np.array(output).reshape(-1)[0])
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        all_preds.append(pred)
        all_probs.append(prob)
        all_labels.append(label)

    if len(set(all_labels)) < 2:
        auc = float('nan')
    else:
        auc = roc_auc_score(all_labels, all_probs)

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()

    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, all_filenames, all_preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc

    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt

    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'],
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'],
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1],
                   tflite_metrics[2], tflite_metrics[3],
                   tflite_metrics[4]]

    x = np.arange(len(metrics))
    width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')

    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# MAIN TRAINING LOOP
# =========================
if __name__ == "__main__":
    resolution = RESOLUTION
    results = []
    cv_index_records = []

    print(f"\n===== Processing right resolution: {resolution} =====")
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    labels_np = np.array(strat_train_data['label'])

    fold = 1
    for train_idx, val_idx in kf.split(np.zeros_like(labels_np), labels_np):
        print(f"\n--- right Fold {fold} ---")
        cv_index_records.append({"fold": fold, "train_indices": train_idx.tolist(), "val_indices": val_idx.tolist()})

        train_subset = {k: [v[i] for i in train_idx] for k, v in strat_train_data.items()}
        val_subset   = {k: [v[i] for i in val_idx]   for k, v in strat_train_data.items()}

        train_loader = make_loader(TrirightDataset(train_subset, train_transform), BATCH_CV, True)
        val_loader   = make_loader(TrirightDataset(val_subset,   test_transform),  BATCH_CV, False)

        model = TriResNetright().to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=LR_CV)
        scaler = GradScaler(enabled=(USE_AMP and device.type == "cuda"))

        for epoch in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in train_loader:
                x1, x2, x3 = [img.to(device).float() for img in imgs]
                labels = labels.to(device).float().unsqueeze(1)
                optimizer.zero_grad(set_to_none=True)
                with autocast(enabled=(USE_AMP and device.type == "cuda")):
                    out = model(x1, x2, x3)
                    loss = criterion(out, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_loss += loss.item()
            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{EPOCHS_CV}] Loss: {total_loss:.6f}")

        val_metrics = evaluate_with_predictions(model, val_loader, val_subset['filenames'])
        result_row = {
            'EyeSet': 'right',
            'Resolution': resolution,
            'Fold': fold,
            'Val_Precision': val_metrics[0],
            'Val_Recall': val_metrics[1],
            'Val_F1': val_metrics[2],
            'Val_Accuracy': val_metrics[3],
            'Val_AUC': val_metrics[4],
            'Val_TP': val_metrics[5],
            'Val_TN': val_metrics[6],
            'Val_FP': val_metrics[7],
            'Val_FN': val_metrics[8]
        }
        results.append(result_row)
        print(results)
        if SAVE_EVERY_FOLD_MODEL:
            fold_path = os.path.join(output_dir, f"right_cv_fold_{fold}_res{resolution}.pt")
            torch.save({'model_state': model.state_dict()}, fold_path)

        fold += 1

    # Save CV results
    pd.DataFrame(results).to_csv(os.path.join(output_dir, "right_val_cross_validation_results.csv"), index=False)
    with open(os.path.join(output_dir, "right_cv_indices.json"), "w") as f:
        json.dump(cv_index_records, f, indent=2)

    # Select best fold
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision','Val_Recall']].min(axis=1)
    candidates = df[(df['Val_Precision'] >= 0.90) & (df['Val_Recall'] >= 0.90)]
    best = candidates.sort_values(['Val_F1'], ascending=False).iloc[0] if len(candidates) > 0 else \
           df.sort_values(['minPR','Val_F1'], ascending=False).iloc[0]
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold}")

    # Load best model
    ckpt_path = os.path.join(output_dir, f"right_cv_fold_{best_fold}_res{resolution}.pt")
    best_model = TriResNetright().to(device)
    state = torch.load(ckpt_path, map_location=device)
    best_model.load_state_dict(state['model_state'])

    # Test evaluation (PyTorch) - Random Stratified Test Set
    test_loader_strat = make_loader(TrirightDataset(strat_test_data, test_transform), BATCH_CV, False)
    test_metrics_strat = evaluate_with_predictions(best_model, test_loader_strat, strat_test_data['filenames'])

    test_results_df = pd.DataFrame([{
        'ChosenFold': best_fold,
        'Test_Precision': test_metrics_strat[0],
        'Test_Recall': test_metrics_strat[1],
        'Test_F1': test_metrics_strat[2],
        'Test_Accuracy': test_metrics_strat[3],
        'Test_AUC': test_metrics_strat[4],
        'Test_TP': test_metrics_strat[5],
        'Test_TN': test_metrics_strat[6],
        'Test_FP': test_metrics_strat[7],
        'Test_FN': test_metrics_strat[8],
        'Test_Type': 'Random_Stratified'
    }])
    test_results_df.to_csv(os.path.join(output_dir, "right_bestfold_test_results_stratified.csv"), index=False)

    print("\n📊 TEST Results (Random Stratified Test Set - Shared Backbone):")
    print(test_results_df.to_string(index=False))

    # Save detailed PyTorch predictions to CSV
    save_predictions_to_csv(
        test_metrics_strat[11],  # filenames
        test_metrics_strat[9],   # true labels
        test_metrics_strat[12],  # pred labels
        test_metrics_strat[10],  # pred probs
        os.path.join(output_dir, "detailed_predictions_pytorch.csv")
    )

    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics_strat[9], test_metrics_strat[10],
                   "ROC Curve - PyTorch Model (Random Stratified Test Set)",
                   os.path.join(output_dir, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics_strat[9],
                          test_metrics_strat[12],
                          "Confusion Matrix - PyTorch Model",
                          os.path.join(output_dir, "confusion_matrix_pytorch.png"))

    # Convert to TFLite
    tflite_filename = "tri_right_eye_resnet18_shared_stratified.tflite"
    tflite_path = convert_to_tflite(best_model.cpu(), output_dir, resolution, tflite_filename)

    if tflite_path:
        size_mb = os.path.getsize(tflite_path) / (1024 * 1024)
        print(f"\n🎉 SUCCESS! Final TFLite model size: {size_mb:.2f} MB")
        print(f"📍 Path: {tflite_path}")

        # --- 🔍 Re-evaluate TFLite model on random stratified test set ---
        print("\n🔍 Re-evaluating TFLite model on random stratified test set...")
        try:
            tflite_metrics = evaluate_tflite_model_with_predictions(tflite_path, strat_test_data, resolution)
            tflite_results_df = pd.DataFrame([{
                'Source': 'TFLite',
                'Test_Precision': tflite_metrics[0],
                'Test_Recall': tflite_metrics[1],
                'Test_F1': tflite_metrics[2],
                'Test_Accuracy': tflite_metrics[3],
                'Test_AUC': tflite_metrics[4],
                'Test_TP': tflite_metrics[5],
                'Test_TN': tflite_metrics[6],
                'Test_FP': tflite_metrics[7],
                'Test_FN': tflite_metrics[8],
                'Test_Type': 'Random_Stratified'
            }])

            combined = pd.concat([
                test_results_df.assign(Source='PyTorch'),
                tflite_results_df
            ], ignore_index=True)

            print("\n📊 COMPARISON: PyTorch vs TFLite (Random Stratified Test Set)")
            print(combined.to_string(index=False))
            combined.to_csv(os.path.join(output_dir, "pytorch_vs_tflite_comparison_stratified.csv"), index=False)

            # Save detailed TFLite predictions to CSV
            save_predictions_to_csv(
                tflite_metrics[11],  # filenames
                tflite_metrics[9],   # true labels
                tflite_metrics[12],  # pred labels
                tflite_metrics[10],  # pred probs
                os.path.join(output_dir, "detailed_predictions_tflite.csv")
            )

            # Plot ROC curve and confusion matrix for TFLite model
            plot_roc_curve(tflite_metrics[9], tflite_metrics[10],
                           "ROC Curve - TFLite Model (Random Stratified Test Set)",
                           os.path.join(output_dir, "roc_curve_tflite.png"))
            plot_confusion_matrix(tflite_metrics[9],
                                  tflite_metrics[12],
                                  "Confusion Matrix - TFLite Model",
                                  os.path.join(output_dir, "confusion_matrix_tflite.png"))

            # Create metrics comparison plot
            plot_metrics_comparison(test_results_df.iloc[0].to_dict(), tflite_metrics,
                                   os.path.join(output_dir, "metrics_comparison.png"))

            tol = 1e-3
            if (abs(tflite_metrics[2] - test_metrics_strat[2]) < tol and
                abs(tflite_metrics[4] - test_metrics_strat[4]) < tol):
                print("✅ TFLite results MATCH PyTorch within tolerance.")
            else:
                print("⚠️ WARNING: TFLite results differ significantly from PyTorch!")
        except Exception as e:
            print(f"❌ TFLite evaluation failed: {e}")
    else:
        print("❌ TFLite conversion failed.")

    print("\n✅ Pipeline completed. Model evaluated on random stratified test set with shared backbone.")
    print(f"✅ Detailed prediction CSVs and plots saved to: {output_dir}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3

📂 TEST  anemic (right)
right1  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=43
right2  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=40
right3  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=40

📂 TEST  non-anemic (right)
right1  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/ | files=322
right2  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemi

INFO:absl:Function `__call__` contains input name(s) input, x, y with unsupported characters which will be renamed to onnx_tf_prefix_identity_49_input, transpose_187_x, onnx_tf_prefix__fusion_fusion_1_add_1_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared_stratified/tri_right_tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared_stratified/tri_right_tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared_stratified/tri_right_tf_model/fingerprint.pb


   ✅ TF SavedModel saved: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared_stratified/tri_right_tf_model
3. TF -> TFLite (SELECT_TF_OPS)...


2025-11-18 07:56:01.385865: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-18 07:56:01.385897: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-18 07:56:01.387503: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared_stratified/tri_right_tf_model
2025-11-18 07:56:01.423259: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-18 07:56:01.423282: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared_stratified/tri_right_tf_model
2025-11-18 07:56:01.438857: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
20

   ✅ TFLite saved: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared_stratified/tri_right_eye_resnet18_shared_stratified.tflite (45.92 MB)

🎉 SUCCESS! Final TFLite model size: 45.92 MB
📍 Path: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared_stratified/tri_right_eye_resnet18_shared_stratified.tflite

🔍 Re-evaluating TFLite model on random stratified test set...


2025-11-18 07:56:02.659680: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-18 07:56:02.660962: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


🔍 TFLite input names: ['serving_default_input1:0', 'serving_default_input2:0', 'serving_default_input3:0']
   ➤ Detected layout: NCHW, size: 224x224

📊 COMPARISON: PyTorch vs TFLite (Random Stratified Test Set)
 ChosenFold  Test_Precision  Test_Recall  Test_F1  Test_Accuracy  Test_AUC  Test_TP  Test_TN  Test_FP  Test_FN         Test_Type  Source
        2.0            0.95         0.95     0.95       0.988604  0.999116       38      309        2        2 Random_Stratified PyTorch
        NaN            0.95         0.95     0.95       0.988604  0.999116       38      309        2        2 Random_Stratified  TFLite
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared_stratified/detailed_predictions_tflite.csv
✅ TFLite results MATCH PyTorch within tolerance.

✅ Pipeline completed. Model evaluated on random stratified test set with shared backbone.
✅ Detailed prediction CSVs and plots saved 

In [ ]:
"done"

'done'

In [ ]:
# -*- coding: utf-8 -*-
"""
Reproducible Hexa-Eye ResNet18 with SHARED BACKBONE + EARLY STOPPING & TFLite Verification
----------------------------------------------------------------------------------
✅ Single ResNet18 used across 6 inputs → ~47 MB TFLite
✅ Stops fold early if Val Precision & Recall >= 0.90
✅ Re-evaluates .tflite model and compares predictions with PyTorch
✅ Ensures faithful deployment (no numerical drift)
✅ Deterministic training + FlexDelegate support for GELU
✅ Added random stratified test split for robustness evaluation
✅ Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
✅ Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison
"""

import os
import json
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold, train_test_split
import warnings

warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 224
EPOCHS_CV = 300
BATCH_CV = 8  # Reduced due to 6 inputs per sample
LR_CV = 0.0002


# 🔥 NEW: Early stop if both P and R >= this threshold
EARLY_STOP_PR = 0.90

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

try:
    cv2.setNumThreads(0)
except Exception:
    pass

os.environ["PYTHONHASHSEED"] = str(SEED)
set_global_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    try:
        print("GPU:", torch.cuda.get_device_name(0))
    except Exception:
        pass

# =========================
# PATHS
# =========================
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
output_dir = os.path.join(base_path, "hexa_eye_hb_90_repro_bestfold_only_sharedf_stratified")
os.makedirs(output_dir, exist_ok=True)

def make_full_path(subdirs):
    return {k: os.path.join(base_path, v) for k, v in subdirs.items()}

# All six input directories
train_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/'
})
train_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/'
})
val_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/'
})
val_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/'
})
test_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'
})
test_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'
})

# =========================
# UTILS
# =========================
def base_from(fname, suffix):
    return fname[:-len(suffix)] if fname.endswith(suffix) else None

def common_bases_hexa(dirs_map):
    suffixes = {
        'left1': '_left_eye_1.png', 'left2': '_left_eye_2.png', 'left3': '_left_eye_3.png',
        'right1': '_right_eye_1.png', 'right2': '_right_eye_2.png', 'right3': '_right_eye_3.png'
    }
    bases_sets = []
    for k in dirs_map.keys():
        folder = dirs_map[k]
        if not os.path.isdir(folder):
            return []
        names = [f for f in os.listdir(folder) if f.endswith(suffixes[k])]
        bases = {base_from(f, suffixes[k]) for f in names if base_from(f, suffixes[k]) is not None}
        bases_sets.append(bases)
    if not bases_sets:
        return []
    inter = set.intersection(*bases_sets)
    return sorted(inter)

def load_hexa_images_by_bases_with_filenames(dirs_map, bases):
    out = {f'r{i}': [] for i in range(1,7)}
    out['filenames'] = []
    key_map = [
        ('left1', '_left_eye_1.png'),
        ('left2', '_left_eye_2.png'),
        ('left3', '_left_eye_3.png'),
        ('right1', '_right_eye_1.png'),
        ('right2', '_right_eye_2.png'),
        ('right3', '_right_eye_3.png')
    ]
    for b in bases:
        imgs, failed = [], False
        for long_k, suf in key_map:
            path = os.path.join(dirs_map[long_k], b + suf)
            if not os.path.isfile(path):
                failed = True
                break
            img = cv2.imread(path)
            if img is None:
                failed = True
                break
            imgs.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if not failed:
            for i, img in enumerate(imgs):
                out[f'r{i+1}'].append(img)
            out['filenames'].append(b)  # Use base name as identifier
    return out

def prepare_dataset_hexa_with_filenames(anemic_dirs, non_dirs, split_name="(split)"):
    bases_a = common_bases_hexa(anemic_dirs)
    bases_n = common_bases_hexa(non_dirs)
    imgs_a = load_hexa_images_by_bases_with_filenames(anemic_dirs, bases_a)
    imgs_n = load_hexa_images_by_bases_with_filenames(non_dirs, bases_n)

    data = {
        'r1': imgs_a['r1'] + imgs_n['r1'],
        'r2': imgs_a['r2'] + imgs_n['r2'],
        'r3': imgs_a['r3'] + imgs_n['r3'],
        'r4': imgs_a['r4'] + imgs_n['r4'],
        'r5': imgs_a['r5'] + imgs_n['r5'],
        'r6': imgs_a['r6'] + imgs_n['r6'],
        'filenames': imgs_a['filenames'] + imgs_n['filenames'],
        'label': [1]*len(imgs_a['r1']) + [0]*len(imgs_n['r1'])
    }
    print(f"✅ {split_name}: anemic={len(imgs_a['r1'])}, non-anemic={len(imgs_n['r1'])}, total={len(data['label'])}")
    try:
        df_bases = pd.DataFrame({'class': ['anemic']*len(bases_a) + ['non_anemic']*len(bases_n),
                                 'base_id': bases_a + bases_n})
        df_bases.to_csv(os.path.join(output_dir, f"{split_name.lower()}_used_base_ids_hexa.csv"), index=False)
    except Exception:
        pass
    return data

def count_files(d):
    return sum(1 for f in sorted(os.listdir(d)) if f.endswith(".png")) if os.path.isdir(d) else 0

def print_dir_stats(title, dirs_map):
    print(f"\n📂 {title}")
    for k in dirs_map.keys():
        p = dirs_map[k]; c = count_files(p)
        print(f"{k:7s} | {p} | files={c}")

# =========================
# LOAD DATA
# =========================

print_dir_stats("TEST  anemic (HEXA)", test_dirs_anemic)
print_dir_stats("TEST  non-anemic (HEXA)", test_dirs_non)

train_data = prepare_dataset_hexa_with_filenames(train_dirs_anemic, train_dirs_non, split_name="TRAIN")
val_data   = prepare_dataset_hexa_with_filenames(val_dirs_anemic,   val_dirs_non,   split_name="VAL")
test_data  = prepare_dataset_hexa_with_filenames(test_dirs_anemic,  test_dirs_non,  split_name="TEST")

if len(train_data['label']) == 0:
    raise RuntimeError("No hexa-eye TRAIN samples found.")

# =========================
# CREATE RANDOM STRATIFIED SPLIT (Training + Validation + Test)
# =========================
print("\n--- Creating Random Stratified Split (Training + Validation + Test) ---")
# Combine all data for stratified split
all_r1 = train_data['r1'] + val_data['r1'] + test_data['r1']
all_r2 = train_data['r2'] + val_data['r2'] + test_data['r2']
all_r3 = train_data['r3'] + val_data['r3'] + test_data['r3']
all_r4 = train_data['r4'] + val_data['r4'] + test_data['r4']
all_r5 = train_data['r5'] + val_data['r5'] + test_data['r5']
all_r6 = train_data['r6'] + val_data['r6'] + test_data['r6']
all_filenames = train_data['filenames'] + val_data['filenames'] + test_data['filenames']
all_labels = train_data['label'] + val_data['label'] + test_data['label']

# Create stratified random splits - 75% train, 12.5% val, 12.5% test
X_temp_r1, X_test_strat_r1, X_temp_r2, X_test_strat_r2, X_temp_r3, X_test_strat_r3, X_temp_r4, X_test_strat_r4, X_temp_r5, X_test_strat_r5, X_temp_r6, X_test_strat_r6, fname_temp, fname_test_strat, y_temp, y_test_strat = train_test_split(
    all_r1, all_r2, all_r3, all_r4, all_r5, all_r6, all_filenames, all_labels,
    test_size=0.125,  # 12.5% for test
    stratify=all_labels,
    random_state=SEED
)

X_train_strat_r1, X_val_strat_r1, X_train_strat_r2, X_val_strat_r2, X_train_strat_r3, X_val_strat_r3, X_train_strat_r4, X_val_strat_r4, X_train_strat_r5, X_val_strat_r5, X_train_strat_r6, X_val_strat_r6, fname_train_strat, fname_val_strat, y_train_strat, y_val_strat = train_test_split(
    X_temp_r1, X_temp_r2, X_temp_r3, X_temp_r4, X_temp_r5, X_temp_r6, fname_temp, y_temp,
    test_size=0.142857,  # 12.5% of total (14.2857% of remaining 87.5%) for validation
    stratify=y_temp,
    random_state=SEED
)

strat_train_data = {
    'r1': X_train_strat_r1,
    'r2': X_train_strat_r2,
    'r3': X_train_strat_r3,
    'r4': X_train_strat_r4,
    'r5': X_train_strat_r5,
    'r6': X_train_strat_r6,
    'filenames': fname_train_strat,
    'label': y_train_strat
}

strat_val_data = {
    'r1': X_val_strat_r1,
    'r2': X_val_strat_r2,
    'r3': X_val_strat_r3,
    'r4': X_val_strat_r4,
    'r5': X_val_strat_r5,
    'r6': X_val_strat_r6,
    'filenames': fname_val_strat,
    'label': y_val_strat
}

strat_test_data = {
    'r1': X_test_strat_r1,
    'r2': X_test_strat_r2,
    'r3': X_test_strat_r3,
    'r4': X_test_strat_r4,
    'r5': X_test_strat_r5,
    'r6': X_test_strat_r6,
    'filenames': fname_test_strat,
    'label': y_test_strat
}

print(f"Random stratified train set: {sum(y_train_strat)} anemic / {len(y_train_strat)} total")
print(f"Random stratified val set: {sum(y_val_strat)} anemic / {len(y_val_strat)} total")
print(f"Random stratified test set: {sum(y_test_strat)} anemic / {len(y_test_strat)} total")
print(f"Original test set: {sum(test_data['label'])} anemic / {len(test_data['label'])} total")

# =========================
# DATASET
# =========================
class HexaDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
    def __len__(self):
        return len(self.data['label'])
    def __getitem__(self, idx):
        images = [self.data[f'r{i}'][idx] for i in range(1,7)]
        images = [self.transform(img) for img in images]
        label = self.data['label'][idx]
        return images, label

def seed_worker(worker_id):
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    torch.manual_seed(worker_seed)

def make_loader(dataset, batch_size, shuffle):
    g = torch.Generator()
    g.manual_seed(SEED)
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY and (device.type=='cuda'),
        worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
        generator=g,
    )

# =========================
# MODEL: SHARED RESNET18 BACKBONE (6 inputs)
# =========================
class HexaResNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Single shared backbone
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Identity()

        # Fusion head: 6*512 → 1
        self.fusion = nn.Sequential(
            nn.Linear(6 * 512, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(0.4),
            nn.Linear(512, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(0.25),
            nn.Linear(128, 1)
        )

    def forward(self, x1, x2, x3, x4, x5, x6):
        f1 = self.backbone(x1)
        f2 = self.backbone(x2)
        f3 = self.backbone(x3)
        f4 = self.backbone(x4)
        f5 = self.backbone(x5)
        f6 = self.backbone(x6)
        x = torch.cat([f1, f2, f3, f4, f5, f6], dim=1)
        return self.fusion(x)

# =========================
# EVALUATION (PyTorch) WITH PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []
    all_filenames = []

    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])

    for imgs, labels in loader:
        x_list = [img.to(device).float() for img in imgs]
        labels = labels.to(device).float().unsqueeze(1)
        out = model(*x_list)
        prob = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (prob > 0.5).astype(int)
        all_preds.extend(pred.tolist())
        all_probs.extend(prob.tolist())
        all_labels.extend(labels.cpu().numpy().flatten().tolist())

    if len(all_labels) == 0 or len(set(all_labels)) < 2:
        return [float('nan')] * 9 + [all_labels, all_probs, all_filenames, all_preds]

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, all_filenames, all_preds

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(best_model: nn.Module, output_dir: str, resolution: int, tflite_filename: str):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    best_model.eval().to('cpu')
    dummy_inputs = tuple(torch.randn(1, 3, resolution, resolution) for _ in range(6))
    onnx_path = os.path.join(output_dir, "hexa_model.onnx")
    tf_path = os.path.join(output_dir, "hexa_tf_model")
    tflite_path = os.path.join(output_dir, tflite_filename)

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # Step 1: PyTorch -> ONNX
    print("1. Converting to ONNX...")
    try:
        torch.onnx.export(
            best_model,
            dummy_inputs,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=[f'input{i}' for i in range(1,7)],
            output_names=['output']
        )
        print(f"   ✅ ONNX saved: {onnx_path}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

    # Step 2: ONNX -> TensorFlow
    print("2. ONNX -> TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        if os.path.exists(tf_path):
            import shutil
            shutil.rmtree(tf_path)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TF SavedModel saved: {tf_path}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

    # Step 3: TF -> TFLite with SELECT_TF_OPS
    print("3. TF -> TFLite (SELECT_TF_OPS)...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        tflite_model = converter.convert()
        with open(tflite_path, "wb") as f:
            f.write(tflite_model)
        size_mb = os.path.getsize(tflite_path) / (1024 * 1024)
        print(f"   ✅ TFLite saved: {tflite_path} ({size_mb:.2f} MB)")
        return tflite_path
    except Exception as e:
        print(f"❌ Conversion failed: {e}")
        return None

# =========================
# TFLITE RE-EVALUATION WITH PREDICTIONS
# =========================
def evaluate_tflite_model_with_predictions(tflite_path, test_data, resolution):
    import tensorflow as tf
    from PIL import Image
    import numpy as np
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Sort by name to ensure correct order
    sorted_inputs = sorted(input_details, key=lambda x: x['name'])
    print(f"🔍 TFLite input names: {[d['name'] for d in sorted_inputs]}")

    first_shape = sorted_inputs[0]['shape']
    if len(first_shape) == 4:
        if first_shape[1] == 3:  # [B,C,H,W]
            layout = 'NCHW'
        else:  # [B,H,W,C]
            layout = 'NHWC'

    resize_h, resize_w = int(first_shape[1] if layout == 'NHWC' else first_shape[2]), \
                         int(first_shape[2] if layout == 'NHWC' else first_shape[3])

    print(f"   ➤ Detected layout: {layout}, size: {resize_h}x{resize_w}")

    def preprocess_pil_style(img_rgb):
        img_pil = Image.fromarray(img_rgb)
        img_resized = img_pil.resize((resize_w, resize_h), Image.BILINEAR)
        tensor = to_tensor(img_resized)
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()

    all_preds, all_probs, all_labels, all_filenames = [], [], [], test_data['filenames']

    for i in range(len(test_data['label'])):
        imgs = [test_data[f'r{j}'][i] for j in range(1,7)]
        label = test_data['label'][i]

        for idx, detail in enumerate(sorted_inputs):
            raw_img = imgs[idx]
            processed = preprocess_pil_style(raw_img)

            if layout == 'NCHW':
                model_input = np.expand_dims(processed, axis=0).astype(detail['dtype'])
            else:
                nhwc = np.transpose(processed, (1, 2, 0))
                model_input = np.expand_dims(nhwc, axis=0).astype(detail['dtype'])

            interpreter.set_tensor(detail['index'], model_input)

        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])
        logit = float(np.array(output).reshape(-1)[0])
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        all_preds.append(pred)
        all_probs.append(prob)
        all_labels.append(label)

    if len(set(all_labels)) < 2:
        auc = float('nan')
    else:
        auc = roc_auc_score(all_labels, all_probs)

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()

    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, all_filenames, all_preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc

    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt

    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'],
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'],
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1],
                   tflite_metrics[2], tflite_metrics[3],
                   tflite_metrics[4]]

    x = np.arange(len(metrics))
    width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')

    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# MAIN TRAINING LOOP (with EARLY STOPPING)
# =========================
if __name__ == "__main__":
    resolution = RESOLUTION
    results = []
    cv_index_records = []

    print(f"\n===== Processing HEXA resolution: {resolution} =====")
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    labels_np = np.array(strat_train_data['label'])

    fold = 1
    for train_idx, val_idx in kf.split(np.zeros_like(labels_np), labels_np):
        print(f"\n--- HEXA Fold {fold} ---")
        cv_index_records.append({"fold": fold, "train_indices": train_idx.tolist(), "val_indices": val_idx.tolist()})

        train_subset = {k: [v[i] for i in train_idx] for k, v in strat_train_data.items()}
        val_subset   = {k: [v[i] for i in val_idx]   for k, v in strat_train_data.items()}

        train_loader = make_loader(HexaDataset(train_subset, train_transform), BATCH_CV, True)
        val_loader   = make_loader(HexaDataset(val_subset,   test_transform),  BATCH_CV, False)

        model = HexaResNet().to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=LR_CV)
        scaler = GradScaler(enabled=(USE_AMP and device.type == "cuda"))

        stopped_early = False
        for epoch in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in train_loader:
                x_list = [img.to(device).float() for img in imgs]
                labels = labels.to(device).float().unsqueeze(1)
                optimizer.zero_grad(set_to_none=True)
                with autocast(enabled=(USE_AMP and device.type == "cuda")):
                    out = model(*x_list)
                    loss = criterion(out, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_loss += loss.item()

            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{EPOCHS_CV}] Loss: {total_loss:.6f}")

            # 🔥 EARLY STOPPING CHECK
            if EARLY_STOP_PR is not None:
                val_metrics = evaluate_with_predictions(model, val_loader, val_subset['filenames'])
                p, r = val_metrics[0], val_metrics[1]
                if not (np.isnan(p) or np.isnan(r)) and p >= EARLY_STOP_PR and r >= EARLY_STOP_PR and p < 1 and r < 1:
                    print(f"✅ Early stop at epoch {epoch+1}: P={p:.3f}, R={r:.3f}")
                    stopped_early = True
                    break  # exit inner loop

        # Final evaluation after training (or early stop)
        val_metrics = evaluate_with_predictions(model, val_loader, val_subset['filenames'])
        result_row = {
            'EyeSet': 'HEXA',
            'Resolution': resolution,
            'Fold': fold,
            'Val_Precision': val_metrics[0],
            'Val_Recall': val_metrics[1],
            'Val_F1': val_metrics[2],
            'Val_Accuracy': val_metrics[3],
            'Val_AUC': val_metrics[4],
            'Val_TP': val_metrics[5],
            'Val_TN': val_metrics[6],
            'Val_FP': val_metrics[7],
            'Val_FN': val_metrics[8],
            'Stopped_Early': stopped_early
        }
        results.append(result_row)
        print(results)
        if SAVE_EVERY_FOLD_MODEL:
            fold_path = os.path.join(output_dir, f"hexa_cv_fold_{fold}_res{resolution}.pt")
            torch.save({'model_state': model.state_dict()}, fold_path)

        fold += 1

    # Save CV results
    pd.DataFrame(results).to_csv(os.path.join(output_dir, "hexa_val_cross_validation_results.csv"), index=False)
    with open(os.path.join(output_dir, "hexa_cv_indices.json"), "w") as f:
        json.dump(cv_index_records, f, indent=2)

    # Select best fold
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision','Val_Recall']].min(axis=1)
    candidates = df[(df['Val_Precision'] >= 0.90) & (df['Val_Recall'] >= 0.90) & (df['Val_Precision'] < 1) & (df['Val_Recall'] < 1)]
    best = candidates.sort_values(['Val_F1'], ascending=False).iloc[0] if len(candidates) > 0 else \
           df.sort_values(['minPR','Val_F1'], ascending=False).iloc[0]
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold}")

    # Load best model
    ckpt_path = os.path.join(output_dir, f"hexa_cv_fold_{best_fold}_res{resolution}.pt")
    best_model = HexaResNet().to(device)
    state = torch.load(ckpt_path, map_location=device)
    best_model.load_state_dict(state['model_state'])

    # Test evaluation (Random Stratified Test Set)
    test_loader_strat = make_loader(HexaDataset(strat_test_data, test_transform), BATCH_CV, False)
    test_metrics_strat = evaluate_with_predictions(best_model, test_loader_strat, strat_test_data['filenames'])

    test_results_df = pd.DataFrame([{
        'ChosenFold': best_fold,
        'Test_Precision': test_metrics_strat[0],
        'Test_Recall': test_metrics_strat[1],
        'Test_F1': test_metrics_strat[2],
        'Test_Accuracy': test_metrics_strat[3],
        'Test_AUC': test_metrics_strat[4],
        'Test_TP': test_metrics_strat[5],
        'Test_TN': test_metrics_strat[6],
        'Test_FP': test_metrics_strat[7],
        'Test_FN': test_metrics_strat[8],
        'Test_Type': 'Random_Stratified'
    }])
    test_results_df.to_csv(os.path.join(output_dir, "hexa_bestfold_test_results_stratified.csv"), index=False)

    print("\n📊 TEST Results (Random Stratified Test Set - Shared Backbone):")
    print(test_results_df.to_string(index=False))

    # Save detailed PyTorch predictions to CSV
    save_predictions_to_csv(
        test_metrics_strat[11],  # filenames
        test_metrics_strat[9],   # true labels
        test_metrics_strat[12],  # pred labels
        test_metrics_strat[10],  # pred probs
        os.path.join(output_dir, "detailed_predictions_pytorch.csv")
    )

    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics_strat[9], test_metrics_strat[10],
                   "ROC Curve - PyTorch Model (Random Stratified Test Set)",
                   os.path.join(output_dir, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics_strat[9],
                          test_metrics_strat[12],
                          "Confusion Matrix - PyTorch Model",
                          os.path.join(output_dir, "confusion_matrix_pytorch.png"))

    # Convert to TFLite
    tflite_filename = "hexa_eye_resnet18_shared_stratified.tflite"
    tflite_path = convert_to_tflite(best_model.cpu(), output_dir, resolution, tflite_filename)

    if tflite_path:
        size_mb = os.path.getsize(tflite_path) / (1024 * 1024)
        print(f"\n🎉 SUCCESS! Final TFLite model size: {size_mb:.2f} MB")
        print(f"📍 Path: {tflite_path}")

        # --- 🔍 Re-evaluate TFLite model on random stratified test set ---
        print("\n🔍 Re-evaluating TFLite model on random stratified test set...")
        try:
            tflite_metrics = evaluate_tflite_model_with_predictions(tflite_path, strat_test_data, resolution)
            tflite_results_df = pd.DataFrame([{
                'Source': 'TFLite',
                'Test_Precision': tflite_metrics[0],
                'Test_Recall': tflite_metrics[1],
                'Test_F1': tflite_metrics[2],
                'Test_Accuracy': tflite_metrics[3],
                'Test_AUC': tflite_metrics[4],
                'Test_TP': tflite_metrics[5],
                'Test_TN': tflite_metrics[6],
                'Test_FP': tflite_metrics[7],
                'Test_FN': tflite_metrics[8],
                'Test_Type': 'Random_Stratified'
            }])

            combined = pd.concat([
                test_results_df.assign(Source='PyTorch'),
                tflite_results_df
            ], ignore_index=True)

            print("\n📊 COMPARISON: PyTorch vs TFLite (Random Stratified Test Set)")
            print(combined.to_string(index=False))
            combined.to_csv(os.path.join(output_dir, "pytorch_vs_tflite_comparison_stratified.csv"), index=False)

            # Save detailed TFLite predictions to CSV
            save_predictions_to_csv(
                tflite_metrics[11],  # filenames
                tflite_metrics[9],   # true labels
                tflite_metrics[12],  # pred labels
                tflite_metrics[10],  # pred probs
                os.path.join(output_dir, "detailed_predictions_tflite.csv")
            )

            # Plot ROC curve and confusion matrix for TFLite model
            plot_roc_curve(tflite_metrics[9], tflite_metrics[10],
                           "ROC Curve - TFLite Model (Random Stratified Test Set)",
                           os.path.join(output_dir, "roc_curve_tflite.png"))
            plot_confusion_matrix(tflite_metrics[9],
                                  tflite_metrics[12],
                                  "Confusion Matrix - TFLite Model",
                                  os.path.join(output_dir, "confusion_matrix_tflite.png"))

            # Create metrics comparison plot
            plot_metrics_comparison(test_results_df.iloc[0].to_dict(), tflite_metrics,
                                   os.path.join(output_dir, "metrics_comparison.png"))

            tol = 1e-3
            if (abs(tflite_metrics[2] - test_metrics_strat[2]) < tol and
                abs(tflite_metrics[4] - test_metrics_strat[4]) < tol):
                print("✅ TFLite results MATCH PyTorch within tolerance.")
            else:
                print("⚠️ WARNING: TFLite results differ significantly from PyTorch!")
        except Exception as e:
            print(f"❌ TFLite evaluation failed: {e}")
    else:
        print("❌ TFLite conversion failed.")

    print("\n✅ Hexa-Eye pipeline completed. Model evaluated on random stratified test set with shared backbone.")
    print(f"✅ Detailed prediction CSVs and plots saved to: {output_dir}")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using device: cuda
GPU: NVIDIA H100 80GB HBM3

📂 TEST  anemic (HEXA)
left1   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=44
left2   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=42
left3   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=43
right1  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=43
right2  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=40
right3  | /home/u

2025-11-18 17:35:29.792568: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-18 17:35:29.829953: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-18 17:35:30.075684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-18 17:35:30.781805: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



--- Starting TFLite Conversion Pipeline ---
1. Converting to ONNX...
   ✅ ONNX saved: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_sharedf_stratified/hexa_model.onnx
2. ONNX -> TensorFlow SavedModel...


2025-11-18 17:35:34.321429: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-18 17:35:34.323238: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO:absl:Function `__call__` contains input name(s) input, x, y with unsupported characters which will be renamed to onnx_tf_prefix_identity_49_input, transpose_373_x, onnx_tf_prefix__fusion_fusion_1_add_1_y in the SavedModel.
INFO:absl:Found untraced functions suc

...

   ✅ TF SavedModel saved: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_sharedf_stratified/hexa_tf_model
3. TF -> TFLite (SELECT_TF_OPS)...


2025-11-18 17:35:44.923160: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-18 17:35:44.923199: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-18 17:35:44.925469: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_sharedf_stratified/hexa_tf_model
2025-11-18 17:35:44.967500: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-18 17:35:44.967531: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_sharedf_stratified/hexa_tf_model
2025-11-18 17:35:45.017977: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is

   ✅ TFLite saved: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_sharedf_stratified/hexa_eye_resnet18_shared_stratified.tflite (48.96 MB)

🎉 SUCCESS! Final TFLite model size: 48.96 MB
📍 Path: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_sharedf_stratified/hexa_eye_resnet18_shared_stratified.tflite

🔍 Re-evaluating TFLite model on random stratified test set...


INFO: Created TensorFlow Lite delegate for select TF ops.
2025-11-18 17:35:47.140218: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-18 17:35:47.141551: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO: TfLiteFlexDelegate delegate: 2 nodes delegated out of 540 nodes with 2 partitions.

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


🔍 TFLite input names: ['serving_default_input1:0', 'serving_default_input2:0', 'serving_default_input3:0', 'serving_default_input4:0', 'serving_default_input5:0', 'serving_default_input6:0']
   ➤ Detected layout: NCHW, size: 224x224

📊 COMPARISON: PyTorch vs TFLite (Random Stratified Test Set)
 ChosenFold  Test_Precision  Test_Recall  Test_F1  Test_Accuracy  Test_AUC  Test_TP  Test_TN  Test_FP  Test_FN         Test_Type  Source
        1.0        0.971429     0.894737 0.931507        0.98494  0.997225       34      293        1        4 Random_Stratified PyTorch
        NaN        0.971429     0.894737 0.931507        0.98494  0.997225       34      293        1        4 Random_Stratified  TFLite
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_sharedf_stratified/detailed_predictions_tflite.csv
✅ TFLite results MATCH PyTorch within tolerance.

✅ Hexa-Eye pipeline completed. Model evaluated on ran